# Create the setup_directories.py file first

In [1]:
# Cell 1: Create setup_directories.py
setup_code = '''# Setup script to create required directories in Kaggle
import os

def setup_kaggle_directories():
    """
    Creates the required directory structure in Kaggle working directory
    """
    # Create data_utils folder
    data_utils_path = '/kaggle/working/data_utils'
    os.makedirs(data_utils_path, exist_ok=True)
    
    # Create output directories for processed patches
    output_dirs = [
        '/kaggle/working/processed_data/train/T1',
        '/kaggle/working/processed_data/train/T2', 
        '/kaggle/working/processed_data/train/label',
        '/kaggle/working/processed_data/val/T1',
        '/kaggle/working/processed_data/val/T2',
        '/kaggle/working/processed_data/val/label',
        '/kaggle/working/processed_data/test/T1',
        '/kaggle/working/processed_data/test/T2',
        '/kaggle/working/processed_data/test/label'
    ]
    
    for dir_path in output_dirs:
        os.makedirs(dir_path, exist_ok=True)
        print(f"Created directory: {dir_path}")
    
    print("✅ All directories created successfully!")
    print("\\nDirectory structure:")
    print("📁 /kaggle/working/")
    print("  └── 📁 data_utils/")
    print("  └── 📁 processed_data/")
    print("      ├── 📁 train/ (T1, T2, label)")
    print("      ├── 📁 val/ (T1, T2, label)")  
    print("      └── 📁 test/ (T1, T2, label)")

if __name__ == "__main__":
    setup_kaggle_directories()
'''

# Write the setup file
with open('/kaggle/working/setup_directories.py', 'w') as f:
    f.write(setup_code)

print("✅ setup_directories.py created!")

✅ setup_directories.py created!


In [2]:
exec(open('/kaggle/working/setup_directories.py').read())

Created directory: /kaggle/working/processed_data/train/T1
Created directory: /kaggle/working/processed_data/train/T2
Created directory: /kaggle/working/processed_data/train/label
Created directory: /kaggle/working/processed_data/val/T1
Created directory: /kaggle/working/processed_data/val/T2
Created directory: /kaggle/working/processed_data/val/label
Created directory: /kaggle/working/processed_data/test/T1
Created directory: /kaggle/working/processed_data/test/T2
Created directory: /kaggle/working/processed_data/test/label
✅ All directories created successfully!

Directory structure:
📁 /kaggle/working/
  └── 📁 data_utils/
  └── 📁 processed_data/
      ├── 📁 train/ (T1, T2, label)
      ├── 📁 val/ (T1, T2, label)
      └── 📁 test/ (T1, T2, label)


# Create data_utils folder and files


In [3]:
# Cell 2: Create data_utils directory
import os
os.makedirs('/kaggle/working/data_utils', exist_ok=True)

# Create __init__.py to make it a package
with open('/kaggle/working/data_utils/__init__.py', 'w') as f:
    f.write('# Data utilities package\n')

print("✅ data_utils package created!")

✅ data_utils package created!


In [4]:
# Cell 3: Create augment.py
# Copy the augment.py code here (from the artifact above)
with open('/kaggle/working/data_utils/augment.py', 'w') as f:
    f.write('''import numpy as np
import random
from PIL import Image
import math
import torch
from torchvision import transforms
import torchvision.transforms.functional as tf

class Augmentation:
    def __init__(self):
        pass
    
    def randomSpaceAugment(self, image, source_size=(256, 256), angle=None, scale=(0.8, 1.25), unoverlap=None):
        """
        Apply random spatial augmentations to a list of images
        
        Args:
            image: List of PIL Images
            source_size: Target size (height, width)
            angle: Rotation angle (if None, random angle is chosen)
            scale: Scale range tuple (min, max)
            unoverlap: Overlap parameter (if None, random cropping is applied)
        """
        if angle is None:
            angle = transforms.RandomRotation.get_params([-180, 180])
        
        if isinstance(angle, list):
            angle = random.choice(angle)
        
        # Rotation
        for i in range(len(image)):
            image[i] = image[i].rotate(angle)
        
        # Horizontal flip
        if random.random() > 0.5:
            for i in range(len(image)):
                image[i] = tf.hflip(image[i])
        
        # Vertical flip  
        if random.random() > 0.5:
            for i in range(len(image)):
                image[i] = tf.vflip(image[i])
        
        img_bias_y_rate = 0.0
        img_bias_x_rate = 0.0
        scale_times = 1.0
        
        # Scale resize
        if random.random() > 0.25:
            img_h, img_w = np.array(image[0]).shape[:2]
            scale_times = torch.empty(1).uniform_(scale[0], scale[1]).item()
            scale_h, scale_w = int(img_h * scale_times), int(img_w * scale_times)
            
            # Resize RGB images (first 2) with BICUBIC
            for i in range(0, min(2, len(image))):
                image[i] = image[i].resize((scale_w, scale_h))
            
            # Resize label images with NEAREST neighbor
            for i in range(2, len(image)):
                image[i] = image[i].resize((scale_w, scale_h), resample=Image.NEAREST)
        
        if unoverlap is None:
            img_h, img_w = np.array(image[0]).shape[:2]
            img_center_x = img_w // 2
            img_center_y = img_h // 2
            
            # Calculate crop boundaries
            angle_rad = math.radians(angle)
            factor = (math.sin(angle_rad) + math.cos(angle_rad)) * scale_times
            
            crop_w_start_img1 = img_center_x - (source_size[1] * factor) // 2
            crop_w_end_img1 = img_center_x + (source_size[1] * factor) // 2 - source_size[1]
            crop_h_start_img1 = img_center_y - (source_size[0] * factor) // 2
            crop_h_end_img1 = img_center_y + (source_size[0] * factor) // 2 - source_size[0]
            
            crop_w_range_img1 = (crop_w_start_img1, crop_w_end_img1)
            crop_h_range_img1 = (crop_h_start_img1, crop_h_end_img1)
            
            crop_w_start_x_img1 = int(torch.empty(1).uniform_(min(crop_w_range_img1), max(crop_w_range_img1)).item())
            crop_h_start_y_img1 = int(torch.empty(1).uniform_(min(crop_h_range_img1), max(crop_h_range_img1)).item())
            
            # Crop all images
            for each_img in range(len(image)):
                image[each_img] = image[each_img].crop((
                    crop_w_start_x_img1,
                    crop_h_start_y_img1, 
                    crop_w_start_x_img1 + source_size[1], 
                    crop_h_start_y_img1 + source_size[0]
                ))
        else:
            # Center crop
            img_h, img_w = np.array(image[0]).shape[:2]
            img_center_x = img_w // 2
            img_center_y = img_h // 2
            crop_w_start_x = img_center_x - source_size[1] // 2
            crop_h_start_y = img_center_y - source_size[0] // 2
            
            for each_img in range(len(image)):
                image[each_img] = image[each_img].crop((
                    crop_w_start_x, 
                    crop_h_start_y, 
                    crop_w_start_x + source_size[1], 
                    crop_h_start_y + source_size[0]
                ))
        
        return image, img_bias_y_rate, img_bias_x_rate

def mirrorPadding2D(image):
    """
    Apply mirror padding to a 2D image
    
    Args:
        image: numpy array of shape (h, w, c)
    
    Returns:
        image_mirror_padding: mirror padded image of shape (2*h, 2*w, c)
    """
    h, w, c = image.shape
    th, dh = h // 2, h - h // 2
    lw, rw = w // 2, w - w // 2
    
    image_mirror_padding = np.zeros((h * 2, w * 2, c), dtype=np.uint8)
    
    # Flip operations
    image_td = np.flip(image, axis=0)
    image_lr = np.flip(image, axis=1)
    image_tdlr = np.flip(image, axis=(0, 1))
    
    # Place original image in center
    image_mirror_padding[th:(th + h), lw:(lw + w)] = image
    
    # Fill corners
    image_mirror_padding[0:th, 0:lw] = image_tdlr[dh:, rw:]
    image_mirror_padding[0:th, (lw + w):] = image_tdlr[dh:, 0:rw]
    image_mirror_padding[(dh + h):, 0:lw] = image_tdlr[0:dh, rw:]
    image_mirror_padding[(dh + h):, (lw + w):] = image_tdlr[0:dh, 0:rw]
    
    # Fill edges
    image_mirror_padding[0:th, lw:(lw + w)] = image_td[dh:, :]
    image_mirror_padding[(th + h):, lw:(lw + w)] = image_td[0:dh, :]
    image_mirror_padding[th:(th + h), 0:lw] = image_lr[:, rw:]
    image_mirror_padding[th:(th + h), (lw + w):] = image_lr[:, 0:rw]
    
    return image_mirror_padding''')

print("✅ augment.py created!")

✅ augment.py created!


In [5]:
# Cell 5: Create slicingPatch.py
# Copy the slicingPatch.py code here (from the artifact above)
with open('/kaggle/working/data_utils/slicingPatch.py', 'w') as f:
    f.write('''import os
import sys
from datetime import datetime
import cv2
import numpy as np
import socket
from PIL import Image
import glob
import argparse
import tqdm

try:
    from osgeo import gdal, gdal_array
    GDAL_AVAILABLE = True
except ImportError:
    GDAL_AVAILABLE = False
    print("Warning: GDAL not available. Only PNG/JPG processing will work.")

# Multi-scale factors for slicing
multiScale = [1.0]

def writeTiff(im_data, path, im_bands, im_height, im_width, im_geotrans, im_proj):
    """
    Write image data to TIFF format
    Note: This function requires GDAL which may not be available in Kaggle
    """
    if not GDAL_AVAILABLE:
        raise ImportError("GDAL is required for TIFF operations but not available")
    
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    elif 'int32' in im_data.dtype.name:
        datatype = gdal.GDT_UInt32
    else:
        datatype = gdal.GDT_Float32
    
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape
    
    # Create file
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, im_width, im_height, 1, datatype)
    
    if dataset is not None:
        dataset.SetGeoTransform(im_geotrans)  # Write affine transform parameters
        dataset.SetProjection(im_proj)  # Write projection
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
    del dataset

def readTiff(filename):
    """
    Read TIFF image file
    Note: This function requires GDAL which may not be available in Kaggle
    """
    if not GDAL_AVAILABLE:
        raise ImportError("GDAL is required for TIFF operations but not available")
    
    img_ds = gdal.Open(filename, gdal.GA_ReadOnly)
    im_width = img_ds.RasterXSize  # Number of columns
    im_height = img_ds.RasterYSize  # Number of rows
    im_bands = img_ds.RasterCount  # Number of bands
    im_geotrans = img_ds.GetGeoTransform()  # Affine transformation
    im_proj = img_ds.GetProjection()  # Map projection info
    
    im_data = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount), 
                       gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
    
    for b in range(im_data.shape[2]):
        im_data[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    
    return im_data, img_ds

def getAxisBoundary(Index, length, imgsize, totalNums):
    """
    Calculate boundary indices for slicing
    
    Args:
        Index: Current index
        length: Total length of the dimension
        imgsize: Size of each slice
        totalNums: Total number of slices
    
    Returns:
        tuple: (start, end) indices
    """
    if (Index + 1 == totalNums) and (length % imgsize != 0):
        start = length - imgsize
        end = length
    else:
        start = Index * imgsize
        end = (Index + 1) * imgsize
    return start, end

def slicingSingleImg(imgDir, outputDir, imgsize=512, scales=[1.0], isLabel=False):
    """
    Slice a single large image into smaller patches
    
    Args:
        imgDir: Path to input image
        outputDir: Path to output directory
        imgsize: Size of output patches
        scales: List of scale factors
        isLabel: Whether the image is a label/mask
    """
    path, name = os.path.split(imgDir)
    subpath, datasetName = os.path.split(path)
    _, datasetName = os.path.split(subpath)
    filename, extension = os.path.splitext(name)
    outputPath = os.path.join(outputDir, f"{datasetName}_{filename}")
    
    save_extension = ".png"
    
    try:
        if extension.lower() in [".png", ".jpg", ".jpeg", ".bmp"]:
            sourceImg = Image.open(imgDir)
        elif extension.lower() in [".tiff", ".tif"] and GDAL_AVAILABLE:
            sourceImg, img_ds1 = readTiff(imgDir)
            sourceImg = Image.fromarray(sourceImg) if sourceImg.shape[2] != 1 else Image.fromarray(sourceImg.squeeze(axis=2))
        else:
            print(f"Skipping unsupported format: {extension}")
            return
        
        sourceW, sourceH = sourceImg.size[0], sourceImg.size[1]
        sourceC = len(sourceImg.getbands())
        
        for singleScale in scales:
            # Determine resampling method based on image type
            resample = Image.NEAREST if isLabel else Image.BICUBIC
            img = sourceImg.resize((int(sourceW * singleScale), int(sourceH * singleScale)), resample=resample)
            img = np.array(img)
            
            # Process labels
            if isLabel:
                img[img != 0] = 255
                img = img.astype(np.uint8)
                
                # Convert RGB label to binary if needed
                if sourceC != 1 and isLabel:
                    img = img.max(axis=2)
            
            img_h, img_w = img.shape[0], img.shape[1]
            h_nums, w_nums = img.shape[0] // imgsize, img.shape[1] // imgsize
            
            if img_h % imgsize != 0:
                h_nums = h_nums + 1
            if img_w % imgsize != 0:
                w_nums = w_nums + 1
            
            if img_h < imgsize or img_w < imgsize:
                continue
            
            print(f"Processing scale {singleScale}: {h_nums}x{w_nums} patches")
            
            for hIndex in tqdm.tqdm(range(h_nums), desc=f"Processing {filename}"):
                start_h, end_h = getAxisBoundary(hIndex, img_h, imgsize, h_nums)
                
                for wIndex in range(w_nums):
                    start_w, end_w = getAxisBoundary(wIndex, img_w, imgsize, w_nums)
                    
                    # Generate output filename
                    outputPathTemp = (f"{outputPath}_scale-{singleScale}_y-{start_h}"
                                    f"_x-{start_w}_imgsize-{imgsize}{save_extension}")
                    
                    if os.path.exists(outputPathTemp):
                        continue
                    
                    # Extract patch
                    if sourceC != 1 and not isLabel:
                        temp_img = img[start_h:end_h, start_w:end_w, :]
                    else:
                        temp_img = img[start_h:end_h, start_w:end_w]
                    
                    # Save patch
                    temp_img = Image.fromarray(temp_img)
                    temp_img.save(outputPathTemp)
    
    except Exception as e:
        print(f"Error processing {imgDir}: {str(e)}")

def process_levir_dataset(input_path, output_path, imgsize=256):
    """
    Process the entire LEVIR-CD dataset
    
    Args:
        input_path: Path to input dataset (/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256)
        output_path: Path to output directory (/kaggle/working/processed_data)
        imgsize: Size of output patches
    """
    splits = ['train', 'val', 'test']
    image_types = ['T1', 'T2', 'label']
    
    for split in splits:
        print(f"\n🔄 Processing {split} split...")
        
        for img_type in image_types:
            input_dir = os.path.join(input_path, split, img_type)
            output_dir = os.path.join(output_path, split, img_type)
            
            if not os.path.exists(input_dir):
                print(f"⚠️  Input directory not found: {input_dir}")
                continue
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir, exist_ok=True)
            
            # Get all files in input directory
            fileList = glob.glob(os.path.join(input_dir, "*"))
            
            if not fileList:
                print(f"⚠️  No files found in {input_dir}")
                continue
            
            print(f"📁 Processing {img_type}: {len(fileList)} files")
            
            isLabel = (img_type == 'label')
            count = 0
            
            for singleImg in fileList:
                count += 1
                slicingSingleImg(
                    singleImg, 
                    output_dir, 
                    scales=multiScale, 
                    imgsize=imgsize, 
                    isLabel=isLabel
                )
                
                if count % 10 == 0:  # Progress update every 10 files
                    print(f"    Processed {count}/{len(fileList)} files")
            
            print(f"✅ Completed {img_type}: {count}/{len(fileList)} files processed")

def main():
    """Main function for command line usage"""
    parser = argparse.ArgumentParser(description='Slice large images into smaller patches for change detection')
    parser.add_argument("-i", "--input_dir", 
                       default="/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256", 
                       type=str, help="Path to input dataset directory")
    parser.add_argument("-o", "--output_dir", 
                       default="/kaggle/working/processed_data", 
                       type=str, help="Path to output directory")
    parser.add_argument("-is", "--img_size", default=256, type=int, help="Output patch size")
    parser.add_argument("-ol", "--overlap_size", default=512, type=int, help="Overlap size (not implemented)")
    parser.add_argument('-c', "--multi_scale_slicing", action='store_true', default=False)
    parser.add_argument('-l', "--is_label", action='store_true', default=False)
    parser.add_argument('--process_full_dataset', action='store_true', default=False, 
                       help="Process the entire LEVIR dataset structure")
    
    args = parser.parse_args()
    
    if args.output_dir is None:
        print("Error: No output directory specified!")
        exit(0)
    
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    if args.process_full_dataset:
        # Process entire LEVIR dataset
        process_levir_dataset(args.input_dir, args.output_dir, args.img_size)
    else:
        # Process single directory (original functionality)
        fileList = glob.glob(os.path.join(args.input_dir, "*"))
        print(f"Input directory: {args.input_dir}")
        print(f"Output directory: {args.output_dir}")
        print(f"Is label: {args.is_label}")
        print(f"Files to process: {len(fileList)}")
        
        count = 0
        for singleImg in fileList:
            count += 1
            slicingSingleImg(
                singleImg, 
                args.output_dir, 
                scales=multiScale, 
                imgsize=args.img_size, 
                isLabel=args.is_label
            )
            print(f"Processed {count}/{len(fileList)}")

if __name__ == "__main__":
    main()''')

print("✅ slicingPatch.py created!")

✅ slicingPatch.py created!


- Implementing slicingPatch.py

In [ ]:
import os
import sys
from datetime import datetime
import cv2
import numpy as np
import socket
from PIL import Image
import glob
import argparse
import tqdm

try:
    from osgeo import gdal, gdal_array
    GDAL_AVAILABLE = True
except ImportError:
    GDAL_AVAILABLE = False
    print("Warning: GDAL not available. Only PNG/JPG processing will work.")

# Multi-scale factors for slicing
multiScale = [1.0]

def writeTiff(im_data, path, im_bands, im_height, im_width, im_geotrans, im_proj):
    """
    Write image data to TIFF format
    Note: This function requires GDAL which may not be available in Kaggle
    """
    if not GDAL_AVAILABLE:
        raise ImportError("GDAL is required for TIFF operations but not available")
    
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    elif 'int32' in im_data.dtype.name:
        datatype = gdal.GDT_UInt32
    else:
        datatype = gdal.GDT_Float32
    
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape
    
    # Create file
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, im_width, im_height, 1, datatype)
    
    if dataset is not None:
        dataset.SetGeoTransform(im_geotrans)  # Write affine transform parameters
        dataset.SetProjection(im_proj)  # Write projection
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
    del dataset

def readTiff(filename):
    """
    Read TIFF image file
    Note: This function requires GDAL which may not be available in Kaggle
    """
    if not GDAL_AVAILABLE:
        raise ImportError("GDAL is required for TIFF operations but not available")
    
    img_ds = gdal.Open(filename, gdal.GA_ReadOnly)
    im_width = img_ds.RasterXSize  # Number of columns
    im_height = img_ds.RasterYSize  # Number of rows
    im_bands = img_ds.RasterCount  # Number of bands
    im_geotrans = img_ds.GetGeoTransform()  # Affine transformation
    im_proj = img_ds.GetProjection()  # Map projection info
    
    im_data = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount), 
                       gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
    
    for b in range(im_data.shape[2]):
        im_data[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    
    return im_data, img_ds

def getAxisBoundary(Index, length, imgsize, totalNums):
    """
    Calculate boundary indices for slicing
    
    Args:
        Index: Current index
        length: Total length of the dimension
        imgsize: Size of each slice
        totalNums: Total number of slices
    
    Returns:
        tuple: (start, end) indices
    """
    if (Index + 1 == totalNums) and (length % imgsize != 0):
        start = length - imgsize
        end = length
    else:
        start = Index * imgsize
        end = (Index + 1) * imgsize
    return start, end

def slicingSingleImg(imgDir, outputDir, imgsize=512, scales=[1.0], isLabel=False):
    """
    Slice a single large image into smaller patches
    
    Args:
        imgDir: Path to input image
        outputDir: Path to output directory
        imgsize: Size of output patches
        scales: List of scale factors
        isLabel: Whether the image is a label/mask
    """
    path, name = os.path.split(imgDir)
    subpath, datasetName = os.path.split(path)
    _, datasetName = os.path.split(subpath)
    filename, extension = os.path.splitext(name)
    outputPath = os.path.join(outputDir, f"{datasetName}_{filename}")
    
    save_extension = ".png"
    
    try:
        if extension.lower() in [".png", ".jpg", ".jpeg", ".bmp"]:
            sourceImg = Image.open(imgDir)
        elif extension.lower() in [".tiff", ".tif"] and GDAL_AVAILABLE:
            sourceImg, img_ds1 = readTiff(imgDir)
            sourceImg = Image.fromarray(sourceImg) if sourceImg.shape[2] != 1 else Image.fromarray(sourceImg.squeeze(axis=2))
        else:
            print(f"Skipping unsupported format: {extension}")
            return
        
        sourceW, sourceH = sourceImg.size[0], sourceImg.size[1]
        sourceC = len(sourceImg.getbands())
        
        for singleScale in scales:
            # Determine resampling method based on image type
            resample = Image.NEAREST if isLabel else Image.BICUBIC
            img = sourceImg.resize((int(sourceW * singleScale), int(sourceH * singleScale)), resample=resample)
            img = np.array(img)
            
            # Process labels
            if isLabel:
                img[img != 0] = 255
                img = img.astype(np.uint8)
                
                # Convert RGB label to binary if needed
                if sourceC != 1 and isLabel:
                    img = img.max(axis=2)
            
            img_h, img_w = img.shape[0], img.shape[1]
            h_nums, w_nums = img.shape[0] // imgsize, img.shape[1] // imgsize
            
            if img_h % imgsize != 0:
                h_nums = h_nums + 1
            if img_w % imgsize != 0:
                w_nums = w_nums + 1
            
            if img_h < imgsize or img_w < imgsize:
                continue
            
            print(f"Processing scale {singleScale}: {h_nums}x{w_nums} patches")
            
            for hIndex in tqdm.tqdm(range(h_nums), desc=f"Processing {filename}"):
                start_h, end_h = getAxisBoundary(hIndex, img_h, imgsize, h_nums)
                
                for wIndex in range(w_nums):
                    start_w, end_w = getAxisBoundary(wIndex, img_w, imgsize, w_nums)
                    
                    # Generate output filename
                    outputPathTemp = (f"{outputPath}_scale-{singleScale}_y-{start_h}"
                                    f"_x-{start_w}_imgsize-{imgsize}{save_extension}")
                    
                    if os.path.exists(outputPathTemp):
                        continue
                    
                    # Extract patch
                    if sourceC != 1 and not isLabel:
                        temp_img = img[start_h:end_h, start_w:end_w, :]
                    else:
                        temp_img = img[start_h:end_h, start_w:end_w]
                    
                    # Save patch
                    temp_img = Image.fromarray(temp_img)
                    temp_img.save(outputPathTemp)
    
    except Exception as e:
        print(f"Error processing {imgDir}: {str(e)}")

def process_levir_dataset(input_path, output_path, imgsize=256):
    """
    Process the entire LEVIR-CD dataset
    
    Args:
        input_path: Path to input dataset (/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256)
        output_path: Path to output directory (/kaggle/working/processed_data)
        imgsize: Size of output patches
    """
    splits = ['train', 'val', 'test']
    image_types = ['T1', 'T2', 'label']
    
    for split in splits:
        print(f"\n🔄 Processing {split} split...")
        
        for img_type in image_types:
            input_dir = os.path.join(input_path, split, img_type)
            output_dir = os.path.join(output_path, split, img_type)
            
            if not os.path.exists(input_dir):
                print(f"⚠️  Input directory not found: {input_dir}")
                continue
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir, exist_ok=True)
            
            # Get all files in input directory
            fileList = glob.glob(os.path.join(input_dir, "*"))
            
            if not fileList:
                print(f"⚠️  No files found in {input_dir}")
                continue
            
            print(f"📁 Processing {img_type}: {len(fileList)} files")
            
            isLabel = (img_type == 'label')
            count = 0
            
            for singleImg in fileList:
                count += 1
                slicingSingleImg(
                    singleImg, 
                    output_dir, 
                    scales=multiScale, 
                    imgsize=imgsize, 
                    isLabel=isLabel
                )
                
                if count % 10 == 0:  # Progress update every 10 files
                    print(f"    Processed {count}/{len(fileList)} files")
            
            print(f"✅ Completed {img_type}: {count}/{len(fileList)} files processed")

def main():
    """Main function for command line usage"""
    parser = argparse.ArgumentParser(description='Slice large images into smaller patches for change detection')
    parser.add_argument("-i", "--input_dir", 
                       default="/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256", 
                       type=str, help="Path to input dataset directory")
    parser.add_argument("-o", "--output_dir", 
                       default="/kaggle/working/processed_data", 
                       type=str, help="Path to output directory")
    parser.add_argument("-is", "--img_size", default=256, type=int, help="Output patch size")
    parser.add_argument("-ol", "--overlap_size", default=512, type=int, help="Overlap size (not implemented)")
    parser.add_argument('-c', "--multi_scale_slicing", action='store_true', default=False)

    args, unknown = parser.parse_known_args()
    
    if args.output_dir is None:
        print("Error: No output directory specified!")
        exit(0)
    
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    # Always process full dataset
    process_levir_dataset(args.input_dir, args.output_dir, args.img_size)

if __name__ == "__main__":
    main()

In [6]:
# Cell 1: Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU count: 2
GPU 0: Tesla T4
GPU 1: Tesla T4


In [7]:
import os
import sys
from datetime import datetime
import cv2
import numpy as np
import socket
from PIL import Image
import glob
import argparse
import tqdm
import gc
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

try:
    import torch
    import torch.nn.functional as F
    GPU_AVAILABLE = torch.cuda.is_available()
    if GPU_AVAILABLE:
        print(f"✅ PyTorch GPU available")
        print(f"🔢 Number of GPUs detected: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"🖥️  GPU {i}: {gpu_name} ({gpu_memory:.1f}GB)")
    else:
        print("⚠️ PyTorch available but no GPU detected")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ PyTorch not available. Install with: pip install torch")

try:
    from osgeo import gdal, gdal_array
    GDAL_AVAILABLE = True
except ImportError:
    GDAL_AVAILABLE = False
    print("Warning: GDAL not available. Only PNG/JPG processing will work.")

# Multi-scale factors for slicing
multiScale = [1.0]

# T4 optimized settings
T4_OPTIMAL_BATCH_SIZE = 32  # Optimized for T4's memory bandwidth
T4_MAX_IMAGE_SIZE = 4096    # Maximum image size to process in single GPU operation

def setup_gpu_devices():
    """Setup and configure available GPUs"""
    if not GPU_AVAILABLE:
        return []
    
    devices = []
    for i in range(torch.cuda.device_count()):
        gpu_props = torch.cuda.get_device_properties(i)
        
        # Get memory info
        torch.cuda.set_device(i)
        torch.cuda.empty_cache()
        
        total_memory = gpu_props.total_memory / (1024**3)  # GB
        allocated_memory = torch.cuda.memory_allocated(i) / (1024**3)  # GB
        free_memory = total_memory - allocated_memory
        
        devices.append({
            'id': i,
            'name': gpu_props.name,
            'free_memory': free_memory,
            'total_memory': total_memory,
            'compute_capability': f"{gpu_props.major}.{gpu_props.minor}"
        })
        
        print(f"🖥️  GPU {i}: {gpu_props.name}")
        print(f"   💾 Memory: {free_memory:.1f}GB free / {total_memory:.1f}GB total")
        print(f"   🔧 Compute Capability: {gpu_props.major}.{gpu_props.minor}")
    
    return devices

def clear_gpu_memory(device_id=None):
    """Clear GPU memory to prevent OOM errors"""
    if GPU_AVAILABLE:
        if device_id is not None:
            torch.cuda.set_device(device_id)
            torch.cuda.empty_cache()
        else:
            # Clear all devices
            for i in range(torch.cuda.device_count()):
                torch.cuda.set_device(i)
                torch.cuda.empty_cache()
    gc.collect()

def writeTiff(im_data, path, im_bands, im_height, im_width, im_geotrans, im_proj):
    """
    Write image data to TIFF format
    Note: This function requires GDAL which may not be available in Kaggle
    """
    if not GDAL_AVAILABLE:
        raise ImportError("GDAL is required for TIFF operations but not available")
    
    # Convert from GPU to CPU if needed
    if GPU_AVAILABLE and torch.is_tensor(im_data):
        im_data = im_data.cpu().numpy()
    
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    elif 'int32' in im_data.dtype.name:
        datatype = gdal.GDT_UInt32
    else:
        datatype = gdal.GDT_Float32
    
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape
    
    # Create file
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, im_width, im_height, 1, datatype)
    
    if dataset is not None:
        dataset.SetGeoTransform(im_geotrans)
        dataset.SetProjection(im_proj)
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
    del dataset

def readTiff(filename):
    """
    Read TIFF image file
    Note: This function requires GDAL which may not be available in Kaggle
    """
    if not GDAL_AVAILABLE:
        raise ImportError("GDAL is required for TIFF operations but not available")
    
    img_ds = gdal.Open(filename, gdal.GA_ReadOnly)
    im_width = img_ds.RasterXSize
    im_height = img_ds.RasterYSize
    im_bands = img_ds.RasterCount
    im_geotrans = img_ds.GetGeoTransform()
    im_proj = img_ds.GetProjection()
    
    im_data = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount), 
                       gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
    
    for b in range(im_data.shape[2]):
        im_data[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    
    return im_data, img_ds

def torch_resize_image_t4_optimized(img_array, new_size, is_label=False, device_id=0):
    """
    T4-optimized GPU image resize using PyTorch
    
    Args:
        img_array: Input image as numpy array
        new_size: (width, height) tuple for new size
        is_label: Whether this is a label image (use nearest neighbor)
        device_id: GPU device ID to use
    
    Returns:
        Resized image array as numpy
    """
    if not GPU_AVAILABLE:
        # CPU fallback using OpenCV
        interpolation = cv2.INTER_NEAREST if is_label else cv2.INTER_CUBIC
        return cv2.resize(img_array, new_size, interpolation=interpolation)
    
    try:
        device = torch.device(f'cuda:{device_id}')
        torch.cuda.set_device(device_id)
        
        # For very large images, use tiled processing
        if max(img_array.shape[:2]) > T4_MAX_IMAGE_SIZE:
            return torch_resize_large_image(img_array, new_size, is_label, device_id)
        
        # Convert numpy to torch tensor
        if len(img_array.shape) == 3:
            # Convert HWC to NCHW format for PyTorch
            img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).unsqueeze(0).float()
        else:
            # Single channel: HW to NCHW
            img_tensor = torch.from_numpy(img_array).unsqueeze(0).unsqueeze(0).float()
        
        # Transfer to GPU
        img_tensor = img_tensor.to(device)
        
        # Resize using PyTorch
        mode = 'nearest' if is_label else 'bicubic'
        
        # PyTorch expects (N, C, H, W) and size as (H, W)
        resized_tensor = F.interpolate(
            img_tensor, 
            size=(new_size[1], new_size[0]),  # PyTorch expects (H, W)
            mode=mode,
            align_corners=False if mode == 'bicubic' else None
        )
        
        # Convert back to numpy
        if len(img_array.shape) == 3:
            # Convert NCHW back to HWC
            result = resized_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
        else:
            # Convert NCHW back to HW
            result = resized_tensor.squeeze(0).squeeze(0).cpu().numpy()
        
        # Clean up GPU memory
        del img_tensor, resized_tensor
        torch.cuda.empty_cache()
        
        # Ensure correct data type
        if is_label:
            result = result.astype(np.uint8)
        else:
            result = result.astype(img_array.dtype)
        
        return result
        
    except Exception as e:
        print(f"⚠️ T4 GPU resize failed on device {device_id}, falling back to CPU: {e}")
        clear_gpu_memory(device_id)
        interpolation = cv2.INTER_NEAREST if is_label else cv2.INTER_CUBIC
        return cv2.resize(img_array, new_size, interpolation=interpolation)

def torch_resize_large_image(img_array, new_size, is_label=False, device_id=0):
    """
    Handle very large images by processing in tiles using PyTorch
    """
    tile_size = T4_MAX_IMAGE_SIZE // 2
    overlap = 64
    
    # Calculate scale factors
    scale_h = new_size[1] / img_array.shape[0]
    scale_w = new_size[0] / img_array.shape[1]
    
    # Create output array
    if len(img_array.shape) == 3:
        output = np.zeros((new_size[1], new_size[0], img_array.shape[2]), dtype=img_array.dtype)
    else:
        output = np.zeros((new_size[1], new_size[0]), dtype=img_array.dtype)
    
    for y in range(0, img_array.shape[0], tile_size - overlap):
        for x in range(0, img_array.shape[1], tile_size - overlap):
            # Extract tile
            y_end = min(y + tile_size, img_array.shape[0])
            x_end = min(x + tile_size, img_array.shape[1])
            tile = img_array[y:y_end, x:x_end]
            
            # Calculate output coordinates
            out_y = int(y * scale_h)
            out_x = int(x * scale_w)
            out_y_end = int(y_end * scale_h)
            out_x_end = int(x_end * scale_w)
            
            # Resize tile
            tile_new_size = (out_x_end - out_x, out_y_end - out_y)
            resized_tile = torch_resize_image_t4_optimized(tile, tile_new_size, is_label, device_id)
            
            # Place in output
            if len(img_array.shape) == 3:
                output[out_y:out_y_end, out_x:out_x_end, :] = resized_tile
            else:
                output[out_y:out_y_end, out_x:out_x_end] = resized_tile
    
    return output

def torch_process_patches_dual_t4(img_array, patch_coords, imgsize, is_label=False, batch_size=32):
    """
    Process patches using dual T4 GPUs for maximum performance with PyTorch
    
    Args:
        img_array: Input image array
        patch_coords: List of (start_h, end_h, start_w, end_w) coordinates
        imgsize: Size of patches
        is_label: Whether processing labels
        batch_size: Batch size per GPU
    
    Returns:
        List of patch arrays
    """
    if not GPU_AVAILABLE or len(patch_coords) < batch_size:
        # Fallback to CPU for small batches
        return cpu_process_patches(img_array, patch_coords, is_label)
    
    num_gpus = torch.cuda.device_count()
    
    if num_gpus == 1:
        return torch_process_patches_single(img_array, patch_coords, imgsize, is_label, 0, batch_size)
    
    # Dual GPU processing
    patches = [None] * len(patch_coords)
    
    def process_gpu_batch(gpu_id, coords_batch, indices_batch):
        try:
            device = torch.device(f'cuda:{gpu_id}')
            torch.cuda.set_device(gpu_id)
            
            # Convert image to tensor once per batch
            if len(img_array.shape) == 3:
                # HWC to CHW
                img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).float().to(device)
            else:
                # HW to CHW (add channel dimension)
                img_tensor = torch.from_numpy(img_array).unsqueeze(0).float().to(device)
            
            for i, (start_h, end_h, start_w, end_w) in enumerate(coords_batch):
                if len(img_array.shape) == 3 and not is_label:
                    # Multi-channel patch: CHW format
                    patch_tensor = img_tensor[:, start_h:end_h, start_w:end_w]
                    # Convert back to HWC for saving
                    cpu_patch = patch_tensor.permute(1, 2, 0).cpu().numpy()
                else:
                    # Single channel patch
                    if len(img_array.shape) == 3:
                        # Take first channel for labels
                        patch_tensor = img_tensor[0, start_h:end_h, start_w:end_w]
                    else:
                        patch_tensor = img_tensor[0, start_h:end_h, start_w:end_w]
                    cpu_patch = patch_tensor.cpu().numpy()
                
                # Ensure correct data type
                if is_label:
                    cpu_patch = cpu_patch.astype(np.uint8)
                else:
                    cpu_patch = cpu_patch.astype(img_array.dtype)
                
                patches[indices_batch[i]] = cpu_patch
            
            # Clean up GPU memory
            del img_tensor
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"⚠️ GPU {gpu_id} batch processing failed: {e}")
            # Fallback to CPU for this batch
            for i, (start_h, end_h, start_w, end_w) in enumerate(coords_batch):
                if len(img_array.shape) == 3 and not is_label:
                    patch = img_array[start_h:end_h, start_w:end_w, :]
                else:
                    patch = img_array[start_h:end_h, start_w:end_w]
                patches[indices_batch[i]] = patch
    
    # Split work between GPUs
    total_batches = len(patch_coords)
    batches_per_gpu = total_batches // num_gpus
    
    with ThreadPoolExecutor(max_workers=num_gpus) as executor:
        futures = []
        
        for gpu_id in range(num_gpus):
            start_idx = gpu_id * batches_per_gpu
            if gpu_id == num_gpus - 1:  # Last GPU gets remaining work
                end_idx = total_batches
            else:
                end_idx = start_idx + batches_per_gpu
            
            coords_batch = patch_coords[start_idx:end_idx]
            indices_batch = list(range(start_idx, end_idx))
            
            future = executor.submit(process_gpu_batch, gpu_id, coords_batch, indices_batch)
            futures.append(future)
        
        # Wait for all GPUs to complete
        for future in as_completed(futures):
            future.result()
    
    return [p for p in patches if p is not None]

def torch_process_patches_single(img_array, patch_coords, imgsize, is_label, device_id, batch_size):
    """Process patches on a single GPU using PyTorch"""
    try:
        device = torch.device(f'cuda:{device_id}')
        torch.cuda.set_device(device_id)
        
        # Convert image to tensor
        if len(img_array.shape) == 3:
            img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).float().to(device)
        else:
            img_tensor = torch.from_numpy(img_array).unsqueeze(0).float().to(device)
        
        patches = []
        
        for start_h, end_h, start_w, end_w in patch_coords:
            if len(img_array.shape) == 3 and not is_label:
                patch_tensor = img_tensor[:, start_h:end_h, start_w:end_w]
                cpu_patch = patch_tensor.permute(1, 2, 0).cpu().numpy()
            else:
                if len(img_array.shape) == 3:
                    patch_tensor = img_tensor[0, start_h:end_h, start_w:end_w]
                else:
                    patch_tensor = img_tensor[0, start_h:end_h, start_w:end_w]
                cpu_patch = patch_tensor.cpu().numpy()
            
            if is_label:
                cpu_patch = cpu_patch.astype(np.uint8)
            else:
                cpu_patch = cpu_patch.astype(img_array.dtype)
            
            patches.append(cpu_patch)
        
        # Clean up
        del img_tensor
        return patches
        
    except Exception as e:
        print(f"⚠️ Single GPU processing failed: {e}")
        return cpu_process_patches(img_array, patch_coords, is_label)

def cpu_process_patches(img_array, patch_coords, is_label):
    """CPU fallback for patch processing"""
    patches = []
    for start_h, end_h, start_w, end_w in patch_coords:
        if len(img_array.shape) == 3 and not is_label:
            patch = img_array[start_h:end_h, start_w:end_w, :]
        else:
            patch = img_array[start_h:end_h, start_w:end_w]
        patches.append(patch)
    return patches

def getAxisBoundary(Index, length, imgsize, totalNums):
    """
    Calculate boundary indices for slicing
    
    Args:
        Index: Current index
        length: Total length of the dimension
        imgsize: Size of each slice
        totalNums: Total number of slices
    
    Returns:
        tuple: (start, end) indices
    """
    if (Index + 1 == totalNums) and (length % imgsize != 0):
        start = length - imgsize
        end = length
    else:
        start = Index * imgsize
        end = (Index + 1) * imgsize
    return start, end

def slicingSingleImg(imgDir, outputDir, imgsize=512, scales=[1.0], isLabel=False, batch_size=T4_OPTIMAL_BATCH_SIZE):
    """
    Slice a single large image into smaller patches using dual T4 GPU acceleration with PyTorch
    
    Args:
        imgDir: Path to input image
        outputDir: Path to output directory
        imgsize: Size of output patches
        scales: List of scale factors
        isLabel: Whether the image is a label/mask
        batch_size: Number of patches to process in GPU batch (optimized for T4)
    """
    path, name = os.path.split(imgDir)
    subpath, datasetName = os.path.split(path)
    _, datasetName = os.path.split(subpath)
    filename, extension = os.path.splitext(name)
    outputPath = os.path.join(outputDir, f"{datasetName}_{filename}")
    
    save_extension = ".png"
    
    try:
        if extension.lower() in [".png", ".jpg", ".jpeg", ".bmp"]:
            sourceImg = Image.open(imgDir)
        elif extension.lower() in [".tiff", ".tif"] and GDAL_AVAILABLE:
            sourceImg, img_ds1 = readTiff(imgDir)
            sourceImg = Image.fromarray(sourceImg) if len(sourceImg.shape) == 3 else Image.fromarray(sourceImg.squeeze(axis=2))
        else:
            print(f"⚠️ Skipping unsupported format: {extension}")
            return
        
        sourceW, sourceH = sourceImg.size[0], sourceImg.size[1]
        sourceC = len(sourceImg.getbands())
        
        for singleScale in scales:
            # Convert PIL to numpy for GPU processing
            img_array = np.array(sourceImg)
            
            # Determine optimal GPU for this operation
            device_id = 0
            if GPU_AVAILABLE and torch.cuda.device_count() > 1:
                # Simple load balancing - use GPU with more free memory
                device_id = 0
            
            # GPU-accelerated resize optimized for T4 with PyTorch
            new_size = (int(sourceW * singleScale), int(sourceH * singleScale))
            img = torch_resize_image_t4_optimized(img_array, new_size, isLabel, device_id)
            
            # Process labels on GPU if available
            if isLabel:
                if GPU_AVAILABLE:
                    try:
                        device = torch.device(f'cuda:{device_id}')
                        torch.cuda.set_device(device_id)
                        
                        # Convert to tensor
                        if len(img.shape) == 3:
                            img_tensor = torch.from_numpy(img).to(device)
                        else:
                            img_tensor = torch.from_numpy(img).to(device)
                        
                        # Binary thresholding
                        img_tensor = torch.where(img_tensor != 0, 255, img_tensor).to(torch.uint8)
                        
                        # Convert RGB label to binary if needed
                        if sourceC != 1 and len(img_tensor.shape) == 3:
                            img_tensor = torch.max(img_tensor, dim=2)[0]
                        
                        img = img_tensor.cpu().numpy()
                        
                        # Clean up
                        del img_tensor
                        torch.cuda.empty_cache()
                        
                    except Exception as e:
                        print(f"⚠️ GPU label processing failed, using CPU: {e}")
                        img[img != 0] = 255
                        img = img.astype(np.uint8)
                        if sourceC != 1 and len(img.shape) == 3:
                            img = img.max(axis=2)
                else:
                    img[img != 0] = 255
                    img = img.astype(np.uint8)
                    if sourceC != 1 and len(img.shape) == 3:
                        img = img.max(axis=2)
            
            img_h, img_w = img.shape[0], img.shape[1]
            h_nums, w_nums = img.shape[0] // imgsize, img.shape[1] // imgsize
            
            if img_h % imgsize != 0:
                h_nums = h_nums + 1
            if img_w % imgsize != 0:
                w_nums = w_nums + 1
            
            if img_h < imgsize or img_w < imgsize:
                continue
            
            print(f"⚡ PyTorch T4 Processing scale {singleScale}: {h_nums}x{w_nums} patches")
            
            # Collect patch coordinates for batch processing
            patch_coords = []
            patch_filenames = []
            
            for hIndex in range(h_nums):
                start_h, end_h = getAxisBoundary(hIndex, img_h, imgsize, h_nums)
                
                for wIndex in range(w_nums):
                    start_w, end_w = getAxisBoundary(wIndex, img_w, imgsize, w_nums)
                    
                    outputPathTemp = (f"{outputPath}_scale-{singleScale}_y-{start_h}"
                                    f"_x-{start_w}_imgsize-{imgsize}{save_extension}")
                    
                    if not os.path.exists(outputPathTemp):
                        patch_coords.append((start_h, end_h, start_w, end_w))
                        patch_filenames.append(outputPathTemp)
            
            # Process patches with dual T4 optimization using PyTorch
            total_patches = len(patch_coords)
            if total_patches == 0:
                continue
                
            print(f"🚀 PyTorch T4 Dual-GPU processing {total_patches} patches...")
            
            # Process all patches at once with dual GPU
            patches = torch_process_patches_dual_t4(img, patch_coords, imgsize, isLabel, batch_size)
            
            # Save patches
            print(f"💾 Saving {len(patches)} patches...")
            for patch, filename in zip(patches, patch_filenames):
                temp_img = Image.fromarray(patch)
                temp_img.save(filename)
            
            # Clear GPU memory after processing
            clear_gpu_memory()
    
    except Exception as e:
        print(f"❌ Error processing {imgDir}: {str(e)}")
        clear_gpu_memory()

def process_levir_dataset(input_path, output_path, imgsize=256, batch_size=T4_OPTIMAL_BATCH_SIZE):
    """
    Process the entire LEVIR-CD dataset with dual T4 GPU acceleration using PyTorch
    
    Args:
        input_path: Path to input dataset
        output_path: Path to output directory
        imgsize: Size of output patches
        batch_size: GPU batch size optimized for T4
    """
    # Setup GPUs
    gpu_devices = setup_gpu_devices()
    
    splits = ['train', 'val', 'test']
    image_types = ['T1', 'T2', 'label']
    
    print(f"🚀 Starting PyTorch T4 Dual-GPU accelerated processing...")
    print(f"🎯 Detected {len(gpu_devices)} GPU(s)")
    print(f"📦 T4-Optimized Batch Size: {batch_size}")
    print(f"🔧 Image Size: {imgsize}")
    
    for split in splits:
        print(f"\n📁 Processing {split} split...")
        
        for img_type in image_types:
            input_dir = os.path.join(input_path, split, img_type)
            output_dir = os.path.join(output_path, split, img_type)
            
            if not os.path.exists(input_dir):
                print(f"⚠️  Input directory not found: {input_dir}")
                continue
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir, exist_ok=True)
            
            # Get all files in input directory
            fileList = glob.glob(os.path.join(input_dir, "*"))
            
            if not fileList:
                print(f"⚠️  No files found in {input_dir}")
                continue
            
            print(f"🖼️  Processing {img_type}: {len(fileList)} files")
            
            isLabel = (img_type == 'label')
            count = 0
            
            for singleImg in tqdm.tqdm(fileList, desc=f"PyTorch T4 Processing {img_type}"):
                count += 1
                slicingSingleImg(
                    singleImg, 
                    output_dir, 
                    scales=multiScale, 
                    imgsize=imgsize, 
                    isLabel=isLabel,
                    batch_size=batch_size
                )
                
                # Clear memory every few files to prevent accumulation
                if count % 3 == 0:
                    clear_gpu_memory()
            
            print(f"✅ Completed {img_type}: {count}/{len(fileList)} files processed")
            clear_gpu_memory()  # Clear memory after each type

def main():
    """Main function for command line usage"""
    parser = argparse.ArgumentParser(description='PyTorch T4 Dual-GPU accelerated image slicing for change detection')
    parser.add_argument("-i", "--input_dir", 
                       default="/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256", 
                       type=str, help="Path to input dataset directory")
    parser.add_argument("-o", "--output_dir", 
                       default="/kaggle/working/processed_data", 
                       type=str, help="Path to output directory")
    parser.add_argument("-is", "--img_size", default=256, type=int, help="Output patch size")
    parser.add_argument("-bs", "--batch_size", default=T4_OPTIMAL_BATCH_SIZE, type=int, 
                       help=f"GPU batch size (T4 optimized default: {T4_OPTIMAL_BATCH_SIZE})")
    parser.add_argument("-ol", "--overlap_size", default=512, type=int, help="Overlap size (not implemented)")
    parser.add_argument('-c', "--multi_scale_slicing", action='store_true', default=False)

    args, unknown = parser.parse_known_args()
    
    if args.output_dir is None:
        print("❌ Error: No output directory specified!")
        exit(0)
    
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    # T4-specific optimizations
    if not GPU_AVAILABLE:
        print("⚠️  Warning: GPU not available, running on CPU")
        args.batch_size = 1
    else:
        print(f"⚡ Using PyTorch with T4 GPU(s), optimized batch size: {args.batch_size}")
        
        # Adjust batch size based on image size for T4's memory
        if args.img_size > 512:
            args.batch_size = max(8, args.batch_size // 2)
            print(f"🔧 Adjusted batch size for large images: {args.batch_size}")
    
    # Process dataset with PyTorch T4 optimizations
    process_levir_dataset(args.input_dir, args.output_dir, args.img_size, args.batch_size)
    
    print("🎉 PyTorch T4 processing completed successfully!")

if __name__ == "__main__":
    main()

✅ PyTorch GPU available
🔢 Number of GPUs detected: 2
🖥️  GPU 0: Tesla T4 (14.7GB)
🖥️  GPU 1: Tesla T4 (14.7GB)
⚡ Using PyTorch with T4 GPU(s), optimized batch size: 32
🖥️  GPU 0: Tesla T4
   💾 Memory: 14.7GB free / 14.7GB total
   🔧 Compute Capability: 7.5
🖥️  GPU 1: Tesla T4
   💾 Memory: 14.7GB free / 14.7GB total
   🔧 Compute Capability: 7.5
🚀 Starting PyTorch T4 Dual-GPU accelerated processing...
🎯 Detected 2 GPU(s)
📦 T4-Optimized Batch Size: 32
🔧 Image Size: 256

📁 Processing train split...
🖼️  Processing T1: 510 files


PyTorch T4 Processing T1:   0%|          | 1/510 [00:00<03:55,  2.16it/s]

⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...
⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...


PyTorch T4 Processing T1:   1%|          | 3/510 [00:00<02:02,  4.14it/s]

⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...
⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...


...

⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...
⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...
⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...


PyTorch T4 Processing label: 100%|█████████▉| 347/348 [00:41<00:00,  9.81it/s]

⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...
⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...
⚡ PyTorch T4 Processing scale 1.0: 1x1 patches
🚀 PyTorch T4 Dual-GPU processing 1 patches...
💾 Saving 1 patches...


PyTorch T4 Processing label: 100%|██████████| 348/348 [00:41<00:00,  8.34it/s]


✅ Completed label: 348/348 files processed
🎉 PyTorch T4 processing completed successfully!


In [8]:
import os

input_dir = "/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256"   # your input folder
output_dir = "/kaggle/working/processed_data"                          # your output folder

def count_files(path):
    total = 0
    for root, dirs, files in os.walk(path):
        total += len([f for f in files if not f.startswith('.')])  # ignore hidden files
    return total

input_files = count_files(input_dir)
output_files = count_files(output_dir)

print(f"📥 Input files: {input_files}")
print(f"📤 Output files: {output_files}")


📥 Input files: 2955
📤 Output files: 2955


In [26]:
import os

output_dir = "/kaggle/working/processed_data"  # change if your output path is different

file_count = 0
for root, dirs, files in os.walk(output_dir):
    file_count += len(files)

print(f"✅ Total processed files saved: {file_count}")


✅ Total processed files saved: 2955


In [9]:
# Cell 4: Create data_loader.py  
# Copy the data_loader.py code here (from the artifact above)
with open('/kaggle/working/data_utils/data_loader.py', 'w') as f:
    f.write('''import glob
import os
import torchvision
from torchvision import transforms
import cv2
import numpy as np
import random
import torch
from torch.utils.data import Dataset
from data_utils.augment import Augmentation, mirrorPadding2D
import pandas as pd
from PIL import Image

class WSIDataset(Dataset):
    def __init__(self, root_dir, mode, taskList=None, total_fold=5, valid_fold=2, miniScale=1):
        """
        Dataset class for LEVIR-CD dataset
        
        Args:
            root_dir: Dictionary or string path to dataset root
            mode: 'train', 'val', or 'test'
            taskList: Not used in current implementation
            total_fold: Total number of folds for cross-validation
            valid_fold: Validation fold number
            miniScale: Scale factor for images
        """
        self.root_dir = root_dir
        self.mode = mode
        self.miniScale = miniScale
        self.total_fold = total_fold
        self.valid_fold = valid_fold
        
        # Initialize lists for image paths
        self.all_png_dir_1 = []  # T1 images
        self.all_png_dir_2 = []  # T2 images  
        self.all_label_change = []  # Labels
        
        # Handle both dictionary and string inputs for root_dir
        if isinstance(root_dir, dict):
            # Original code logic for multiple datasets
            for k, v in self.root_dir.items():
                self.all_png_dir_1 += sorted(glob.glob(self.root_dir[k] + os.sep + self.mode + os.sep + "T1" + os.sep + '*'))
                self.all_png_dir_2 += sorted(glob.glob(self.root_dir[k] + os.sep + self.mode + os.sep + "T2" + os.sep + '*'))
                self.all_label_change += sorted(glob.glob(self.root_dir[k] + os.sep + self.mode + os.sep + "label" + os.sep + '*'))
        else:
            # Single dataset path (for Kaggle usage)
            self.all_png_dir_1 = sorted(glob.glob(os.path.join(root_dir, mode, "T1", '*')))
            self.all_png_dir_2 = sorted(glob.glob(os.path.join(root_dir, mode, "T2", '*')))
            self.all_label_change = sorted(glob.glob(os.path.join(root_dir, mode, "label", '*')))
        
        # Extract filenames for matching
        self.all_png_dir_1_name = [os.path.splitext(os.path.split(i)[1])[0] for i in self.all_label_change]
        
        print(f"T1 patch numbers: {len(self.all_png_dir_1)}")
        print(f"T2 patch numbers: {len(self.all_png_dir_2)}")
        print(f"Label patch numbers: {len(self.all_label_change)}")
        
        # Training parameters
        self.isTrain = False
        self.source_size = (256, 256)
        self.randomImgSizeList = [(256, 256)]
        self.randomImgSizeList = self.randomImgSizeList[::1]
        self.randomImgSize = (256, 256)
    
    def __getitem__(self, index):
        """
        Get a single item from the dataset
        
        Args:
            index: Index of the item to retrieve
            
        Returns:
            tuple: (img1, img2, label1, label2, labelc, dir)
        """
        dir_name = self.all_png_dir_1_name[index]
        img1_path = self.all_png_dir_1[index]
        img2_path = self.all_png_dir_2[index]
        labelc_path = self.all_label_change[index]
        
        if self.mode == "train":
            # Load and resize images for training
            img1 = np.array(Image.open(img1_path).resize(self.randomImgSize))
            img2 = np.array(Image.open(img2_path).resize(self.randomImgSize))
            labelc = np.expand_dims(np.array(Image.open(labelc_path).resize(self.randomImgSize)), axis=2)
            
            # Apply mirror padding for augmentation
            img1 = mirrorPadding2D(img1)
            img2 = mirrorPadding2D(img2)
            labelc = mirrorPadding2D(labelc)
            
            # Convert back to PIL Images
            img1 = Image.fromarray(img1)
            img2 = Image.fromarray(img2)
            labelc = Image.fromarray(np.squeeze(labelc))
            
            # Apply geometric augmentations
            aug = Augmentation()
            img2_combine, bias_y, bias_x = aug.randomSpaceAugment(
                [img1, img2, labelc], 
                source_size=self.randomImgSize, 
                unoverlap=None
            )
            
            # Unpack augmented images
            img1, img2, labelc = img2_combine
            
            # Apply photometric distortions
            imgPhotometricDistortion1 = transforms.Compose([
                transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.2, 0.1)], p=0.8),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ])
            
            imgPhotometricDistortion2 = transforms.Compose([
                transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.2, 0.1)], p=0.8),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ])
            
            img1 = imgPhotometricDistortion1(img1)
            img2 = imgPhotometricDistortion2(img2)
            labelc = torch.FloatTensor(np.array(labelc)) / 255
            
        elif self.mode in ["validation", "val", "test"]:
            # Load and process images for validation/test
            img1 = Image.open(img1_path).resize(self.randomImgSize)
            img2 = Image.open(img2_path).resize(self.randomImgSize)
            
            imgTransforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ])
            
            labelc = np.expand_dims(np.array(Image.open(labelc_path).resize(self.randomImgSize)), axis=2)
            labelc = torch.FloatTensor(np.squeeze(labelc)) / 255
            
            img1 = imgTransforms(img1)
            img2 = imgTransforms(img2)
        
        # Dummy labels (not used in current implementation)
        label1 = torch.FloatTensor([0])
        label2 = torch.FloatTensor([0])
        
        return img1, img2, label1, label2, labelc, dir_name
    
    def __len__(self):
        """Return the total number of samples"""
        return len(self.all_png_dir_1)

if __name__ == "__main__":
    # Example usage
    dataset_path = "/kaggle/working/processed_data"
    
    # Test dataset creation
    train_dataset = WSIDataset(root_dir=dataset_path, mode="train")
    val_dataset = WSIDataset(root_dir=dataset_path, mode="val") 
    test_dataset = WSIDataset(root_dir=dataset_path, mode="test")
    
    print("Dataset created successfully!")
    print(f"Train samples: {len(train_dataset)}")
    print(f"Val samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    # Test loading a single sample
    if len(train_dataset) > 0:
        sample = train_dataset[0]
        print(f"Sample shapes: img1={sample[0].shape}, img2={sample[1].shape}, label={sample[4].shape}")''')

print("✅ data_loader.py created!")

✅ data_loader.py created!


In [10]:
#!/usr/bin/env python3
"""
Quick test script to verify the complete setup works correctly
Run this after setting up all the files to ensure everything is working
"""

import os
import sys
import torch
import torchvision
from PIL import Image
import numpy as np

def test_pytorch_version():
    """Test PyTorch version compatibility"""
    print("🔍 Checking PyTorch version...")
    print(f"   PyTorch version: {torch.__version__}")
    print(f"   Torchvision version: {torchvision.__version__}")
    
    # Check if it's 2.0+
    version_parts = torch.__version__.split('.')
    major_version = int(version_parts[0])
    minor_version = int(version_parts[1]) if len(version_parts) > 1 else 0
    
    if major_version >= 2:
        print("   ✅ PyTorch 2.0+ detected")
        return True
    else:
        print("   ⚠️  PyTorch version is less than 2.0")
        return False

def test_directory_structure():
    """Test if directory structure is correct"""
    print("\n🔍 Checking directory structure...")
    
    required_dirs = [
        '/kaggle/working/data_utils',
        '/kaggle/working/processed_data/train/T1',
        '/kaggle/working/processed_data/train/T2',
        '/kaggle/working/processed_data/train/label',
        '/kaggle/working/processed_data/val/T1',
        '/kaggle/working/processed_data/val/T2',
        '/kaggle/working/processed_data/val/label',
        '/kaggle/working/processed_data/test/T1',
        '/kaggle/working/processed_data/test/T2',
        '/kaggle/working/processed_data/test/label'
    ]
    
    required_files = [
        '/kaggle/working/data_utils/__init__.py',
        '/kaggle/working/data_utils/augment.py',
        '/kaggle/working/data_utils/data_loader.py',
        '/kaggle/working/data_utils/slicingPatch.py'
    ]
    
    all_good = True
    
    for dir_path in required_dirs:
        if os.path.exists(dir_path):
            print(f"   ✅ {dir_path}")
        else:
            print(f"   ❌ {dir_path}")
            all_good = False
    
    for file_path in required_files:
        if os.path.exists(file_path):
            print(f"   ✅ {file_path}")
        else:
            print(f"   ❌ {file_path}")
            all_good = False
    
    return all_good

def test_imports():
    """Test if all imports work correctly"""
    print("\n🔍 Testing imports...")
    
    try:
        # Add to path
        sys.path.append('/kaggle/working')
        
        from data_utils.augment import Augmentation, mirrorPadding2D
        print("   ✅ augment.py imports successful")
        
        from data_utils.data_loader import WSIDataset
        print("   ✅ data_loader.py imports successful")
        
        return True
    except ImportError as e:
        print(f"   ❌ Import error: {e}")
        return False

def test_dataset_loading():
    """Test dataset loading functionality"""
    print("\n🔍 Testing dataset loading...")
    
    # Check if original dataset exists
    dataset_path = "/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256"
    
    if not os.path.exists(dataset_path):
        print(f"   ⚠️  Original dataset not found at {dataset_path}")
        return False
    
    try:
        from data_utils.data_loader import WSIDataset
        
        # Test dataset creation
        train_dataset = WSIDataset(root_dir=dataset_path, mode="train")
        val_dataset = WSIDataset(root_dir=dataset_path, mode="val")
        test_dataset = WSIDataset(root_dir=dataset_path, mode="test")
        
        print(f"   ✅ Datasets created successfully")
        print(f"      Train samples: {len(train_dataset)}")
        print(f"      Val samples: {len(val_dataset)}")
        print(f"      Test samples: {len(test_dataset)}")
        
        # Test loading a sample
        if len(train_dataset) > 0:
            sample = train_dataset[0]
            print(f"   ✅ Sample loaded successfully")
            print(f"      img1 shape: {sample[0].shape}")
            print(f"      img2 shape: {sample[1].shape}")
            print(f"      label shape: {sample[4].shape}")
            print(f"      filename: {sample[5]}")
        
        return True
    except Exception as e:
        print(f"   ❌ Dataset loading error: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_augmentation():
    """Test augmentation functionality"""
    print("\n🔍 Testing augmentation...")
    
    try:
        from data_utils.augment import Augmentation, mirrorPadding2D
        
        # Create dummy images
        img1 = Image.new('RGB', (256, 256), color='red')
        img2 = Image.new('RGB', (256, 256), color='green')
        label = Image.new('L', (256, 256), color='white')
        
        aug = Augmentation()
        
        # Test augmentation
        augmented, bias_y, bias_x = aug.randomSpaceAugment([img1, img2, label])
        
        print("   ✅ Spatial augmentation works")
        
        # Test mirror padding
        test_array = np.random.randint(0, 255, (64, 64, 3), dtype=np.uint8)
        padded = mirrorPadding2D(test_array)
        
        expected_shape = (128, 128, 3)
        if padded.shape == expected_shape:
            print("   ✅ Mirror padding works")
            return True
        else:
            print(f"   ❌ Mirror padding shape mismatch: {padded.shape} != {expected_shape}")
            return False
        
    except Exception as e:
        print(f"   ❌ Augmentation error: {e}")
        return False

def test_dataloader():
    """Test PyTorch DataLoader functionality"""
    print("\n🔍 Testing DataLoader...")
    
    try:
        from data_utils.data_loader import WSIDataset
        from torch.utils.data import DataLoader
        
        dataset_path = "/kaggle/input/processed-levir-cd-dataset/LEVIR-CD+_256"
        
        if not os.path.exists(dataset_path):
            print("   ⚠️  Original dataset not available, skipping DataLoader test")
            return True
        
        # Create dataset and dataloader
        dataset = WSIDataset(root_dir=dataset_path, mode="train")
        
        if len(dataset) == 0:
            print("   ⚠️  No samples in dataset, skipping DataLoader test")
            return True
        
        dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)
        
        # Test loading a batch
        for batch_idx, (img1, img2, label1, label2, change_label, filenames) in enumerate(dataloader):
            print(f"   ✅ DataLoader works")
            print(f"      Batch size: {img1.shape[0]}")
            print(f"      img1 batch shape: {img1.shape}")
            print(f"      img2 batch shape: {img2.shape}")
            print(f"      change_label batch shape: {change_label.shape}")
            break
        
        return True
        
    except Exception as e:
        print(f"   ❌ DataLoader error: {e}")
        return False

def main():
    """Run all tests"""
    print("🧪 Running complete setup verification tests...\n")
    
    tests = [
        ("PyTorch Version", test_pytorch_version),
        ("Directory Structure", test_directory_structure),
        ("Imports", test_imports),
        ("Dataset Loading", test_dataset_loading),
        ("Augmentation", test_augmentation),
        ("DataLoader", test_dataloader)
    ]
    
    results = []
    
    for test_name, test_func in tests:
        try:
            result = test_func()
            results.append((test_name, result))
        except Exception as e:
            print(f"   ❌ {test_name} failed with exception: {e}")
            results.append((test_name, False))
    
    print("\n" + "="*50)
    print("🧪 TEST RESULTS SUMMARY")
    print("="*50)
    
    passed = 0
    total = len(results)
    
    for test_name, result in results:
        status = "✅ PASS" if result else "❌ FAIL"
        print(f"{test_name:20} : {status}")
        if result:
            passed += 1
    
    print("-"*50)
    print(f"TOTAL: {passed}/{total} tests passed")
    
    if passed == total:
        print("🎉 All tests passed! Your setup is ready for use.")
    else:
        print("⚠️  Some tests failed. Please check the errors above.")
    
    return passed == total

if __name__ == "__main__":
    main()

🧪 Running complete setup verification tests...

🔍 Checking PyTorch version...
   PyTorch version: 2.6.0+cu124
   Torchvision version: 0.21.0+cu124
   ✅ PyTorch 2.0+ detected

🔍 Checking directory structure...
   ✅ /kaggle/working/data_utils
   ✅ /kaggle/working/processed_data/train/T1
   ✅ /kaggle/working/processed_data/train/T2
   ✅ /kaggle/working/processed_data/train/label
   ✅ /kaggle/working/processed_data/val/T1
   ✅ /kaggle/working/processed_data/val/T2
   ✅ /kaggle/working/processed_data/val/label
   ✅ /kaggle/working/processed_data/test/T1
   ✅ /kaggle/working/processed_data/test/T2
   ✅ /kaggle/working/processed_data/test/label
   ✅ /kaggle/working/data_utils/__init__.py
   ✅ /kaggle/working/data_utils/augment.py
   ✅ /kaggle/working/data_utils/data_loader.py
   ✅ /kaggle/working/data_utils/slicingPatch.py

🔍 Testing imports...
   ✅ augment.py imports successful
   ✅ data_loader.py imports successful

🔍 Testing dataset loading...
T1 patch numbers: 510
T2 patch numbers: 510
La

# Create the models folder and init.py file

- let's create the models directory and add an __init__.py file to make it a proper Python package.
- Run this script first to create the models folder structure. This will:

    - Create the /kaggle/working/models directory
    - Add an __init__.py file to make it a proper Python package with all the necessary imports

In [11]:
import os

# Create the models directory
models_dir = "/kaggle/working/models"
os.makedirs(models_dir, exist_ok=True)

# Create __init__.py to make it a Python package
init_content = '''"""
Models package for CDNeXt implementation
Contains ConvNeXt, ResNet, and custom layer implementations compatible with PyTorch 2.0
"""

from .cdnext import CDNeXt, get_cdnext
from .convnext import ConvNeXt, convnext_tiny, convnext_small, convnext_base, convnext_large, convnext_xlarge, get_convnext
from .layers import *
from .resnet import ResNet, resnet18, resnet34, resnet50, resnet101, resnet152

__all__ = [
    'CDNeXt', 'get_cdnext',
    'ConvNeXt', 'convnext_tiny', 'convnext_small', 'convnext_base', 
    'convnext_large', 'convnext_xlarge', 'get_convnext',
    'ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'
]
'''

with open(os.path.join(models_dir, "__init__.py"), "w") as f:
    f.write(init_content)

print(f"Created models directory at: {models_dir}")
print("Created __init__.py file")
print("\nNext step: We'll implement layers.py first since other files depend on it.")

Created models directory at: /kaggle/working/models
Created __init__.py file

Next step: We'll implement layers.py first since other files depend on it.


**I've created the layers.py file with all the custom layer classes needed for the CDNeXt implementation. Here are the key improvements I made for PyTorch 2.0 compatibility:**

In [12]:
%%writefile /kaggle/working/models/layers.py

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from einops import rearrange
from timm.models.layers import trunc_normal_, DropPath


class Swish(nn.Module):
    def __init__(self, name=None):
        super().__init__()
        self.name = name

    def forward(self, x):
        return x * torch.sigmoid(x)


class SqueezeDoubleConvOld(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SqueezeDoubleConvOld, self).__init__()
        self.squeeze = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.GELU()
        )
        self.double_conv = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels),
        )
        self.acfun = nn.GELU()

    def forward(self, x):
        x = self.squeeze(x)
        block_x = self.double_conv(x)
        x = self.acfun(x + block_x)
        return x


class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


class CBAM(nn.Module):
    def __init__(self, in_planes, CAon=True, SAon=True, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.CAon = CAon
        self.SAon = SAon
        if self.CAon:
            self.ca = ChannelAttention(in_planes, ratio)
        if self.SAon:
            self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        if self.CAon:
            x = self.ca(x) * x
        if self.SAon:
            x = self.sa(x) * x
        return x


class NonLocal2D(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=2, sub_sample=False, bn_layer=True):
        """
        :param in_channels:
        :param inter_channels:
        :param dimension:
        :param sub_sample:
        :param bn_layer:
        """
        super(NonLocal2D, self).__init__()
        assert dimension in [2]
        self.dimension = dimension
        self.sub_sample = sub_sample
        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
        if self.inter_channels == 0:
            self.inter_channels = 1

        if dimension == 2:
            conv_nd = nn.Conv2d
            max_pool_layer = nn.MaxPool2d(kernel_size=(2, 2))
            bn = nn.BatchNorm2d

        self.g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                         kernel_size=1, stride=1, padding=0)

        self.W = nn.Sequential(
            conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                    kernel_size=1, stride=1, padding=0),
            bn(self.in_channels)
        )

        nn.init.constant_(self.W[1].weight, 0)
        nn.init.constant_(self.W[1].bias, 0)

        self.theta = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(self.inter_channels)
        )
        self.phi = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(self.inter_channels)
        )

    def forward(self, x, return_nl_map=False):
        """
        :param x: (b, c, h, w)
        :param return_nl_map: if True return z, nl_map, else only return z.
        :return:
        """
        batch_size = x.size(0)

        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
        theta_x = theta_x.permute(0, 2, 1)
        phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)

        f = torch.matmul(theta_x, phi_x)
        f_div_C = F.softmax(f, dim=-1)

        y = torch.matmul(f_div_C, g_x)
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])
        W_y = self.W(y)
        z = W_y + x

        if return_nl_map:
            return z, f_div_C
        return z


class SpatiotemporalAttentionFull(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=2, sub_sample=False, bn_layer=True):
        """
        :param in_channels:
        :param inter_channels:
        :param dimension:
        :param sub_sample:
        :param bn_layer:
        """
        super(SpatiotemporalAttentionFull, self).__init__()
        assert dimension in [2]
        self.dimension = dimension
        self.sub_sample = sub_sample
        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
        if self.inter_channels == 0:
            self.inter_channels = 1

        self.g = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0)
        )

        self.W = nn.Sequential(
            nn.Conv2d(in_channels=self.inter_channels, out_channels=self.in_channels,
                     kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(self.in_channels)
        )

        self.theta = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )
        self.phi = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )

        self.energy_time_1_sf = nn.Softmax(dim=-1)
        self.energy_time_2_sf = nn.Softmax(dim=-1)
        self.energy_space_2s_sf = nn.Softmax(dim=-2)
        self.energy_space_1s_sf = nn.Softmax(dim=-2)

    def forward(self, x1, x2):
        """
        :param x1, x2: (b, c, h, w)
        :return: attended features
        """
        batch_size = x1.size(0)

        g_x11 = self.g(x1).reshape(batch_size, self.inter_channels, -1)
        g_x12 = g_x11.permute(0, 2, 1)
        g_x21 = self.g(x2).reshape(batch_size, self.inter_channels, -1)
        g_x22 = g_x21.permute(0, 2, 1)

        theta_x1 = self.theta(x1).reshape(batch_size, self.inter_channels, -1)
        theta_x2 = theta_x1.permute(0, 2, 1)
        phi_x1 = self.phi(x2).reshape(batch_size, self.inter_channels, -1)
        phi_x2 = phi_x1.permute(0, 2, 1)

        energy_time_1 = torch.matmul(theta_x1, phi_x2)
        energy_time_2 = energy_time_1.permute(0, 2, 1)
        energy_space_1 = torch.matmul(theta_x2, phi_x1)
        energy_space_2 = energy_space_1.permute(0, 2, 1)

        energy_time_1s = self.energy_time_1_sf(energy_time_1)
        energy_time_2s = self.energy_time_2_sf(energy_time_2)
        energy_space_2s = self.energy_space_2s_sf(energy_space_1)
        energy_space_1s = self.energy_space_1s_sf(energy_space_2)

        y1 = torch.matmul(torch.matmul(energy_time_2s, g_x11), energy_space_2s).contiguous()
        y2 = torch.matmul(torch.matmul(energy_time_1s, g_x21), energy_space_1s).contiguous()

        y1 = y1.reshape(batch_size, self.inter_channels, *x2.size()[2:])
        y2 = y2.reshape(batch_size, self.inter_channels, *x1.size()[2:])

        return x1 + self.W(y1), x2 + self.W(y2)


class SpatiotemporalAttentionBase(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=2, sub_sample=False, bn_layer=True):
        """
        :param in_channels:
        :param inter_channels:
        :param dimension:
        :param sub_sample:
        :param bn_layer:
        """
        super(SpatiotemporalAttentionBase, self).__init__()
        assert dimension in [2]
        self.dimension = dimension
        self.sub_sample = sub_sample
        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
        if self.inter_channels == 0:
            self.inter_channels = 1

        self.g = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0)
        )

        self.W = nn.Sequential(
            nn.Conv2d(in_channels=self.inter_channels, out_channels=self.in_channels,
                     kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(self.in_channels)
        )

        self.theta = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )
        self.phi = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )

        self.energy_space_2s_sf = nn.Softmax(dim=-2)
        self.energy_space_1s_sf = nn.Softmax(dim=-2)

    def forward(self, x1, x2):
        """
        :param x1, x2: (b, c, h, w)
        :return: attended features
        """
        batch_size = x1.size(0)

        g_x11 = self.g(x1).reshape(batch_size, self.inter_channels, -1)
        g_x21 = self.g(x2).reshape(batch_size, self.inter_channels, -1)

        theta_x1 = self.theta(x1).reshape(batch_size, self.inter_channels, -1)
        theta_x2 = theta_x1.permute(0, 2, 1)
        phi_x1 = self.phi(x2).reshape(batch_size, self.inter_channels, -1)

        energy_space_1 = torch.matmul(theta_x2, phi_x1)
        energy_space_2 = energy_space_1.permute(0, 2, 1)

        energy_space_2s = self.energy_space_2s_sf(energy_space_1)
        energy_space_1s = self.energy_space_1s_sf(energy_space_2)

        y1 = torch.matmul(g_x11, energy_space_2s).contiguous()
        y2 = torch.matmul(g_x21, energy_space_1s).contiguous()

        y1 = y1.reshape(batch_size, self.inter_channels, *x2.size()[2:])
        y2 = y2.reshape(batch_size, self.inter_channels, *x1.size()[2:])

        return x1 + self.W(y1), x2 + self.W(y2)


class SpatiotemporalAttentionFullNotWeightShared(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=2, sub_sample=False, bn_layer=True):
        """
        :param in_channels:
        :param inter_channels:
        :param dimension:
        :param sub_sample:
        :param bn_layer:
        """
        super(SpatiotemporalAttentionFullNotWeightShared, self).__init__()
        assert dimension in [2]
        self.dimension = dimension
        self.sub_sample = sub_sample
        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
        if self.inter_channels == 0:
            self.inter_channels = 1

        self.g1 = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0)
        )
        self.g2 = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )

        self.W1 = nn.Sequential(
            nn.Conv2d(in_channels=self.inter_channels, out_channels=self.in_channels,
                     kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(self.in_channels)
        )
        self.W2 = nn.Sequential(
            nn.Conv2d(in_channels=self.inter_channels, out_channels=self.in_channels,
                     kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(self.in_channels)
        )

        self.theta = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )
        self.phi = nn.Sequential(
            nn.BatchNorm2d(self.in_channels),
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.inter_channels,
                     kernel_size=1, stride=1, padding=0),
        )

    def forward(self, x1, x2):
        """
        :param x1, x2: (b, c, h, w)
        :return: attended features
        """
        batch_size = x1.size(0)

        g_x11 = self.g1(x1).reshape(batch_size, self.inter_channels, -1)
        g_x12 = g_x11.permute(0, 2, 1)
        g_x21 = self.g2(x2).reshape(batch_size, self.inter_channels, -1)
        g_x22 = g_x21.permute(0, 2, 1)

        theta_x1 = self.theta(x1).reshape(batch_size, self.inter_channels, -1)
        theta_x2 = theta_x1.permute(0, 2, 1)
        phi_x1 = self.phi(x2).reshape(batch_size, self.inter_channels, -1)
        phi_x2 = phi_x1.permute(0, 2, 1)

        energy_time_1 = torch.matmul(theta_x1, phi_x2)
        energy_time_2 = energy_time_1.permute(0, 2, 1)
        energy_space_1 = torch.matmul(theta_x2, phi_x1)
        energy_space_2 = energy_space_1.permute(0, 2, 1)

        energy_time_1s = F.softmax(energy_time_1, dim=-1)
        energy_time_2s = F.softmax(energy_time_2, dim=-1)
        energy_space_2s = F.softmax(energy_space_1, dim=-2)
        energy_space_1s = F.softmax(energy_space_2, dim=-2)

        y1 = torch.matmul(torch.matmul(energy_time_2s, g_x11), energy_space_2s).contiguous()
        y2 = torch.matmul(torch.matmul(energy_time_1s, g_x21), energy_space_1s).contiguous()

        y1 = y1.reshape(batch_size, self.inter_channels, *x2.size()[2:])
        y2 = y2.reshape(batch_size, self.inter_channels, *x1.size()[2:])

        return x1 + self.W1(y1), x2 + self.W2(y2)


class DANetModule(nn.Module):
    def __init__(self, in_dim):
        super(DANetModule, self).__init__()
        self.chanel_in = in_dim
        self.sa = PAM_Module(in_dim)
        self.sc = CAM_Module(in_dim)

    def forward(self, x):
        sa_feat = self.sa(x)
        sc_feat = self.sc(x)
        return sa_feat + sc_feat


class PAM_Module(nn.Module):
    """ Position attention module"""
    # Ref from SAGAN
    def __init__(self, in_dim):
        super(PAM_Module, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1),
            nn.BatchNorm2d(in_dim//8),
        )
        self.key_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1),
            nn.BatchNorm2d(in_dim//8),
        )
        self.value_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1),
            nn.BatchNorm2d(in_dim),
        )
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        """
        inputs :
            x : input feature maps( B X C X H X W)
        returns :
            out : attention value + input feature
            attention: B X (HxW) X (HxW)
        """
        m_batchsize, C, height, width = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)
        energy = torch.bmm(proj_query, proj_key)
        attention = self.softmax(energy)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out


class CAM_Module(nn.Module):
    """ Channel attention module"""
    def __init__(self, in_dim):
        super(CAM_Module, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1),
            nn.BatchNorm2d(in_dim//8),
        )
        self.key_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1),
            nn.BatchNorm2d(in_dim//8),
        )
        self.value_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1),
            nn.BatchNorm2d(in_dim),
        )
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        """
        inputs :
            x : input feature maps( B X C X H X W)
        returns :
            out : attention value + input feature
            attention: B X C X C
        """
        m_batchsize, C, height, width = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, C, -1)
        proj_key = self.key_conv(x).view(m_batchsize, C, -1).permute(0, 2, 1)
        energy = torch.bmm(proj_query, proj_key)
        energy_new = torch.max(energy, -1, keepdim=True)[0].expand_as(energy) - energy
        attention = self.softmax(energy_new)
        proj_value = self.value_conv(x).view(m_batchsize, C, -1)

        out = torch.bmm(attention, proj_value)
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out


Writing /kaggle/working/models/layers.py


**I've created the resnet.py file with all the ResNet implementations. Here are the key improvements I made for PyTorch 2.0 compatibility:**

In [14]:
%%writefile /kaggle/working/models/resnet.py

from typing import Type, Any, Callable, Union, List, Optional
import torch
import torch.nn as nn
from torch import Tensor

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

__all__ = [
    "ResNet",
    "resnet18",
    "resnet34",
    "resnet50",
    "resnet101",
    "resnet152",
    "resnext50_32x4d",
    "resnext101_32x8d",
    "wide_resnet50_2",
    "wide_resnet101_2",
]

# The model_urls are copied from https://pytorch.org/docs/1.6.0/_modules/torchvision/models/resnet.html#resnet18
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        strides: List[int] = [1, 1, 2, 2, 2]
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=strides[0], padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=strides[1])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=strides[2], dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=strides[3], dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=strides[4], dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups,
                self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any,
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet("resnet18", BasicBlock, [2, 2, 2, 2], pretrained, progress, **kwargs)


def resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet("resnet34", BasicBlock, [3, 4, 6, 3], pretrained, progress, **kwargs)


def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet("resnet50", Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def resnet101(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet("resnet101", Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)


def resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet("resnet152", Bottleneck, [3, 8, 36, 3], pretrained, progress, **kwargs)


def resnext50_32x4d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs["groups"] = 32
    kwargs["width_per_group"] = 4
    return _resnet("resnext50_32x4d", Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs["groups"] = 32
    kwargs["width_per_group"] = 8
    return _resnet("resnext101_32x8d", Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs["width_per_group"] = 64 * 2
    return _resnet("wide_resnet50_2", Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs["width_per_group"] = 64 * 2
    return _resnet("wide_resnet101_2", Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)

Overwriting /kaggle/working/models/resnet.py


**I've created the convnext.py file with the ConvNeXt implementation. Here are the key improvements I made for PyTorch 2.0 compatibility:**

In [15]:
%%writefile /kaggle/working/models/convnext.py

# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model
from .resnet import resnet18


class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape, )

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)  # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim)  # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                 requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1)  # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x


class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf

    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000,
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0.,
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()

        self.downsample_layers = nn.ModuleList()  # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList()  # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                        layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6)  # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

        self.feature = []

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1]))  # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x


model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
    "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    "convnext_small_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth",
    "convnext_base_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth",
    "convnext_large_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth",
    "convnext_xlarge_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth",
    "convnext_tiny_22k_384": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_1k_384.pth",
    "convnext_small_22k_384": "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_1k_384.pth",
    "convnext_base_22k_384": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth",
    "convnext_large_22k_384": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_1k_384.pth",
    "convnext_xlarge_22k_384": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_1k_384_ema.pth",
}


@register_model
def convnext_tiny(pretrained=False, in_22k=False, classes=1000, resolution=None, **kwargs):
    # 1k is 1000, 22k is 21841
    model = ConvNeXt(num_classes=classes, depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    
    resolution_suffix = "_"+str(resolution) if resolution == 384 else ""
    if pretrained:
        url = model_urls['convnext_tiny_22k'+resolution_suffix] if in_22k else model_urls['convnext_tiny_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        if classes != 1000 and classes != 21841:
            random_state_dict = model.state_dict()
            checkpoint['model']['head.weight'] = random_state_dict['head.weight']
            checkpoint['model']['head.bias'] = random_state_dict['head.bias']
        model.load_state_dict(checkpoint["model"])
    return model


@register_model
def convnext_small(pretrained=False, in_22k=False, classes=1000, resolution=None, **kwargs):
    model = ConvNeXt(num_classes=classes, depths=[3, 3, 27, 3], dims=[96, 192, 384, 768], **kwargs)
    
    resolution_suffix = "_"+str(resolution) if resolution == 384 else ""
    if pretrained:
        url = model_urls['convnext_small_22k'+resolution_suffix] if in_22k else model_urls['convnext_small_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        if classes != 1000 and classes != 21841:
            random_state_dict = model.state_dict()
            checkpoint['model']['head.weight'] = random_state_dict['head.weight']
            checkpoint['model']['head.bias'] = random_state_dict['head.bias']
        model.load_state_dict(checkpoint["model"])
    return model


@register_model
def convnext_base(pretrained=False, in_22k=False, classes=1000, resolution=None, **kwargs):
    model = ConvNeXt(num_classes=classes, depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], **kwargs)
    
    resolution_suffix = "_"+str(resolution) if resolution == 384 else ""
    if pretrained:
        url = model_urls['convnext_base_22k'+resolution_suffix] if in_22k else model_urls['convnext_base_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        if classes != 1000 and classes != 21841:
            random_state_dict = model.state_dict()
            checkpoint['model']['head.weight'] = random_state_dict['head.weight']
            checkpoint['model']['head.bias'] = random_state_dict['head.bias']
        model.load_state_dict(checkpoint["model"])
    return model


@register_model
def convnext_large(pretrained=False, in_22k=False, classes=1000, resolution=None, **kwargs):
    model = ConvNeXt(num_classes=classes, depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
    
    resolution_suffix = "_"+str(resolution) if resolution == 384 else ""
    if pretrained:
        url = model_urls['convnext_large_22k'+resolution_suffix] if in_22k else model_urls['convnext_large_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        if classes != 1000 and classes != 21841:
            random_state_dict = model.state_dict()
            checkpoint['model']['head.weight'] = random_state_dict['head.weight']
            checkpoint['model']['head.bias'] = random_state_dict['head.bias']
        model.load_state_dict(checkpoint["model"])
    return model


@register_model
def convnext_xlarge(pretrained=False, in_22k=False, classes=1000, resolution=None, **kwargs):
    model = ConvNeXt(num_classes=classes, depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048], **kwargs)
    
    resolution_suffix = "_"+str(resolution) if resolution == 384 else ""
    if pretrained:
        assert in_22k, "only ImageNet-22K pre-trained ConvNeXt-XL is available; please set in_22k=True"
        url = model_urls['convnext_xlarge_22k'+resolution_suffix]
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        if classes != 1000 and classes != 21841:
            random_state_dict = model.state_dict()
            checkpoint['model']['head.weight'] = random_state_dict['head.weight']
            checkpoint['model']['head.bias'] = random_state_dict['head.bias']
        model.load_state_dict(checkpoint["model"])
    return model


@register_model
def get_convnext(pretrained=False, backbone_scale='tiny', in_22k=True, classes=1000, resolution=224, **kwargs):
    if backbone_scale == "tiny":
        depths, dims = [3, 3, 9, 3], [96, 192, 384, 768]
    elif backbone_scale == "small":
        depths, dims = [3, 3, 27, 3], [96, 192, 384, 768]
    elif backbone_scale == "base":
        depths, dims = [3, 3, 27, 3], [128, 256, 512, 1024]
    elif backbone_scale == "large":
        depths, dims = [3, 3, 27, 3], [192, 384, 768, 1536]
    elif backbone_scale == "xlarge":
        depths, dims = [3, 3, 27, 3], [256, 512, 1024, 2048]
    elif backbone_scale == "resnet18":
        model = resnet18(pretrained=pretrained, **kwargs)
        return model
    else:
        raise ValueError(f"Unknown backbone_scale: {backbone_scale}")

    model = ConvNeXt(num_classes=classes, depths=depths, dims=dims, **kwargs)
    
    if pretrained:
        clsNums = "22k" if in_22k else "1k"
        resolution_suffix = "_"+str(resolution) if resolution == 384 else ""
        url = model_urls["convnext_"+backbone_scale+"_"+clsNums+resolution_suffix]
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        
        if classes != 1000 and classes != 21841:
            random_state_dict = model.state_dict()
            checkpoint['model']['head.weight'] = random_state_dict['head.weight']
            checkpoint['model']['head.bias'] = random_state_dict['head.bias']
        model.load_state_dict(checkpoint["model"])
    
    return model

Writing /kaggle/working/models/convnext.py


In [16]:
%%writefile /kaggle/working/models/cdnext.py

from collections import OrderedDict
from .layers import *
import copy
from .convnext import convnext_tiny, convnext_small, convnext_base, convnext_large, convnext_xlarge, get_convnext
from .resnet import resnet18
import torch.nn.functional as F
from einops import rearrange

__all__ = ['CDNeXt', 'get_cdnext']


class CDNeXt(nn.Module):
    def __init__(self, encoder, backbone_scale="tiny", out_channels=2, 
                 isTemporalAttention=[1, 2, 3, 4], isCBAM=[0, 0, 0, 0], 
                 isNonlocal=[0, 0, 0, 0], **kwargs):
        super().__init__()
        self.encoder = encoder
        self.isFeatureFusion = True
        self.CAon = False
        self.SAon = False
        self.isCBAMconcat = False
        self.isNonlocalConcat = False
        self.AttentionModule = DANetModule  # NonLocal2D, DANetModule
        
        self.isTemporalAttention = isTemporalAttention
        self.SpatiotemporalAttentionModule = SpatiotemporalAttentionFull  # SpatiotemporalAttentionBase, SpatiotemporalAttentionFull, SpatiotemporalAttentionFullNotWeightShared
        self.isCBAM = isCBAM
        self.isNonlocal = isNonlocal
        self.encoderName = backbone_scale
        
        if "resnet" in self.encoderName:
            self.encoderNameScale = 2
        else:
            self.encoderNameScale = 4
            
        self.AdvSupResult = []
        self.downScale = [16, 8, 4, 0]
        self.stageNumber = 4
        
        self.backbone_depth = {
            'tiny': [3, 3, 9, 3],
            'small': [3, 3, 27, 3],
            'base': [3, 3, 27, 3],
            'large': [3, 3, 27, 3],
            'xlarge': [3, 3, 27, 3],
            "resnet18": [2, 2, 2, 2]
        }
        
        self.backbone_dims = {
            'tiny': [96, 192, 384, 768],
            'small': [96, 192, 384, 768],
            'base': [128, 256, 512, 1024],
            'large': [192, 384, 768, 1536],
            'xlarge': [256, 512, 1024, 2048],
            "resnet18": [64, 128, 256, 512]
        }
        
        self.size_dict = {
            'tiny': [24, 96, 192, 384, 768],
            'small': [24, 96, 192, 384, 768],
            'base': [32, 128, 256, 512, 1024],
            'large': [48, 192, 384, 768, 1536],
            'xlarge': [64, 256, 512, 1024, 2048],
            "resnet18": [32, 64, 128, 256, 512]
        }

        # source constructor
        # init attention module
        self.CBAMs = []
        self.TemporalAttentions = []
        self.Nonlocals = []
        self.ChangeSqueezeConv = []
        
        # module sequence: F.interpolate, TemporalAttention, AdversialSupervised, concat feature, conv
        for index in range(self.stageNumber):
            if index == 0:
                if self.isCBAM[index] != 0:
                    if self.isCBAMconcat:
                        self.CBAMs.append(CBAM(self.n_channels*2, CAon=self.CAon, SAon=self.SAon))
                    else:
                        self.CBAMs.append(CBAM(self.n_channels, CAon=self.CAon, SAon=self.SAon))
                
                if self.isTemporalAttention[index] != 0:
                    self.TemporalAttentions.append(self.SpatiotemporalAttentionModule(self.size_change[index]))
                
                if self.isNonlocal[index] != 0:
                    if self.isNonlocalConcat:
                        self.Nonlocals.append(self.AttentionModule(self.n_channels*2))
                    else:
                        self.Nonlocals.append(self.AttentionModule(self.n_channels))
                
                self.ChangeSqueezeConv.append(SqueezeDoubleConvOld(self.n_channels*2, self.n_channels))
            else:
                if self.isCBAM[index] != 0:
                    if self.isCBAMconcat:
                        self.CBAMs.append(CBAM(self.size_change[index]*2, CAon=self.CAon, SAon=self.SAon))
                    else:
                        self.CBAMs.append(CBAM(self.size_change[index], CAon=self.CAon, SAon=self.SAon))
                
                if self.isTemporalAttention[index] != 0:
                    self.TemporalAttentions.append(self.SpatiotemporalAttentionModule(self.size_change[index]))
                
                if self.isNonlocal[index] != 0:
                    if self.isNonlocalConcat:
                        self.Nonlocals.append(self.AttentionModule(self.size_change[index]*2))
                    else:
                        self.Nonlocals.append(self.AttentionModule(self.size_change[index]))
                
                self.ChangeSqueezeConv.append(SqueezeDoubleConvOld(self.size_change[index]*4, self.size_change[index]))

        self.CBAMs = nn.ModuleList(self.CBAMs)
        self.TemporalAttentions = nn.ModuleList(self.TemporalAttentions)
        self.Nonlocals = nn.ModuleList(self.Nonlocals)
        self.ChangeSqueezeConv = nn.ModuleList(self.ChangeSqueezeConv)

        if self.isFeatureFusion == True:
            self.ChangeFinalSqueezeConv = SqueezeDoubleConvOld(sum(self.size_change[:-1]), self.size_change[-1]*self.encoderNameScale)
            self.ChangeFinalConv = nn.Sequential(
                SqueezeDoubleConvOld(self.size_change[-1]*self.encoderNameScale, self.size_change[-1]),
                nn.Conv2d(self.size_change[-1], out_channels, kernel_size=1)
            )
        else:
            self.ChangeFinalSqueezeConv = SqueezeDoubleConvOld(self.size_change[-2], self.size_change[-1]*self.encoderNameScale)
            self.ChangeFinalConv = nn.Sequential(
                SqueezeDoubleConvOld(self.size_change[-1]*self.encoderNameScale, self.size_change[-1]),
                nn.Conv2d(self.size_change[-1], out_channels, kernel_size=1)
            )

        # self.softmax = nn.Softmax(dim=1)
        self.register_hook(self.encoder)
        self.backboneFeatures = []

    def register_hook(self, backbone):
        if "resnet" in self.encoderName:
            def hook(module, input, output):
                self.backboneFeatures.append(output)
            depth = self.backbone_depth[self.encoderName]
            for index, depth_num in enumerate(depth):
                getattr(backbone, "layer"+str(index+1)).register_forward_hook(hook)
        else:
            def hook(module, input, output):
                self.backboneFeatures.append(output)
            depth = self.backbone_depth[self.encoderName]
            for index, depth_num in enumerate(depth):
                backbone.stages[index][depth_num-1].register_forward_hook(hook)

    @property
    def n_channels(self):
        return self.backbone_dims[self.encoderName][-1]

    @property
    def size_change(self):
        size_dict = copy.deepcopy(self.size_dict)
        size_dict = size_dict[self.encoderName][:-1]
        return size_dict

    def forward(self, x1, x2):
        input_1 = x1
        input_2 = x2
        
        _ = self.encoder(x1)
        _ = self.encoder(x2)
        
        blocks1 = self.backboneFeatures[0:self.stageNumber]
        blocks2 = self.backboneFeatures[self.stageNumber:]
        self.backboneFeatures = []
        
        FusionFeatures = []
        change = None
        
        for stage in range(self.stageNumber):
            moduleIdx = stage
            eff_last_1 = blocks1.pop()  # .permute(0, 3, 1, 2)
            eff_last_2 = blocks2.pop()  # .permute(0, 3, 1, 2)
            
            if self.isTemporalAttention[stage] != 0:
                moduleRealIdx = self.isTemporalAttention[stage] - 1
                eff_last_1, eff_last_2 = self.TemporalAttentions[moduleRealIdx](eff_last_1, eff_last_2)
            
            if self.isNonlocal[stage] != 0:
                moduleIdx = self.isNonlocal[stage] - 1
                if self.isNonlocalConcat:
                    eff_last = self.Nonlocals[moduleIdx](torch.cat([eff_last_1, eff_last_2], dim=1))
                    sliceNum = eff_last.shape[1]//2
                    eff_last_1, eff_last_2 = eff_last[:, 0:sliceNum], eff_last[:, sliceNum:]
                else:
                    eff_last_1, eff_last_2 = self.Nonlocals[moduleIdx](eff_last_1), self.Nonlocals[moduleIdx](eff_last_2)
            
            if self.isCBAM[stage] != 0:
                moduleIdx = self.isCBAM[stage] - 1
                if self.isCBAMconcat:
                    eff_last = self.CBAMs[moduleIdx](torch.cat([eff_last_1, eff_last_2], dim=1))
                    sliceNum = eff_last.shape[1]//2
                    eff_last_1, eff_last_2 = eff_last[:, 0:sliceNum], eff_last[:, sliceNum:]
                else:
                    eff_last_1, eff_last_2 = self.CBAMs[moduleIdx](eff_last_1), self.CBAMs[moduleIdx](eff_last_2)
            
            if stage == 0:
                change = torch.cat([eff_last_1, eff_last_2], dim=1)
            else:
                change = torch.cat([eff_last_1, eff_last_2, change], dim=1)
            
            if stage == self.stageNumber-1:
                change = self.ChangeSqueezeConv[stage](change)
                FusionFeatures.append(change)
            else:
                change = self.ChangeSqueezeConv[stage](change)
                FusionFeatures.append(change)
                change = F.interpolate(change, scale_factor=2., mode='bilinear', align_corners=True)

        if self.isFeatureFusion == True:
            for index, down in enumerate(self.downScale):
                FusionFeatures[index] = F.interpolate(FusionFeatures[index], 
                                                     scale_factor=2**(self.stageNumber-index-1), 
                                                     mode='bilinear', align_corners=True)
            fusion = torch.cat(FusionFeatures, dim=1)
        else:
            fusion = change

        change = self.ChangeFinalSqueezeConv(fusion)
        change = F.interpolate(change, scale_factor=self.encoderNameScale, mode='bilinear', align_corners=True)
        change = self.ChangeFinalConv(change)
        
        return change


def get_cdnext(out_channels=2, backbone_scale="tiny", pretrained=True, in_22k=True, 
               resolution=384, backbone_trainable=False, **kwargs):
    print("is pretrained: ", pretrained)
    encoder = get_convnext(pretrained=pretrained, backbone_scale=backbone_scale, 
                          classes=out_channels, in_22k=in_22k, resolution=resolution, **kwargs)
    model = CDNeXt(encoder, backbone_scale=backbone_scale, out_channels=out_channels, **kwargs)
    
    if "resnet" in backbone_scale:
        pass
    else:
        if backbone_trainable == False:
            for name, value in model.named_parameters():
                if "encoder" in name:
                    value.requires_grad = False
    
    return model

Writing /kaggle/working/models/cdnext.py


In [17]:
%%writefile /kaggle/working/train.py

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import GradScaler
from torch.utils import data as data_
from tqdm import tqdm
from models.cdnext import get_cdnext
from data_utils.data_loader import WSIDataset
import matplotlib.pyplot as plt
from multiprocessing import Process
from prettytable import PrettyTable
import random

labelNameList = ["change"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path setting - updated for Kaggle structure
ROOTDIR = {
    "LEVIRCD": "/kaggle/working/processed_data",
}

backboneName = "tiny"  # 'tiny','small','base','resnet18'
isPretrained = True
SIMULATE_BATCH_SIZE = 32
BATCH_SIZE = 16
accumulate_steps = SIMULATE_BATCH_SIZE // BATCH_SIZE
train_num_workers = 4
test_num_workers = 4
saveDatasetName = "-".join(ROOTDIR.keys())
save_dir = "checkpoint/" + saveDatasetName
total_epoch = 400
use_cuda = True
num_classes = len(labelNameList)
result_dir = "checkpoint/" + saveDatasetName
lossType = "balance ce"
learning_rate = 4e-5
itersDisplayMetrics = ["Acc", "Pre", "Rec", "IoU", "TNR", "Loss"]
epochsDisplayMetrics = ["Acc", "Pre", "Rec", "IoU", "TNR", "F1", "Kappa", "Loss"]
plot_metrics = ["F1", "IoU", "Loss"]
plot_metrics.sort()
stage = ["train", "val"]
stage.sort()
use_amp = True


def setup_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    # torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def data_loader(ROOTDIR, mode="train", taskList=labelNameList,
                miniScale=1, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, 
                total_fold=5, valid_fold=5):
    dataset = WSIDataset(root_dir=ROOTDIR, mode=mode, taskList=taskList, 
                        total_fold=total_fold, valid_fold=valid_fold, miniScale=miniScale)
    dataloader = data_.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=train_num_workers,
        shuffle=shuffle,
        drop_last=drop_last,
        pin_memory=True
    )
    return dataset, dataloader


def save_checkpoints(model, step):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    filename = "CDNet_" + str(step) + "_" + "-".join(labelNameList) + ".pth"
    torch.save(model.state_dict(), os.path.join(save_dir, filename))
    print(f" Save checkpoint {step.split('_')[0]} to {filename}. \n")


def train(model, optimizer, index, trainDataLoader, criterion, scaler, dataSetClass, loss_branch=None):
    showRowLen = num_classes + 1
    tempMetricsList = ["TP", "FP", "TN", "FN", "Acc", "Pre", "Rec", "IoU", "TNR", "Loss"]
    epochMetricsList = ["TP", "FP", "TN", "FN", "Acc", "Pre", "Rec", "IoU", "TNR", "F1", "Kappa", "Loss"]
    
    tempMetrics = {key: [0] * showRowLen for key in tempMetricsList}
    epochMetrics = {key: [0] * showRowLen for key in epochMetricsList}
    
    model.train()
    skipIdx = len(trainDataLoader) // accumulate_steps * accumulate_steps
    
    for batch_idx, (img1, img2, label1, label2, img_change, dir_name) in tqdm(enumerate(trainDataLoader)):
        img1, img2, label1, label2, img_change = img1.float(), img2.float(), label1.float(), label2.float(), img_change.float()
        
        if batch_idx >= skipIdx:
            continue
            
        if use_cuda:
            img1 = img1.to(device)
            img2 = img2.to(device)
            img_change = img_change.to(device)
        
        if scaler is None:
            output_change = model(img1, img2)
            loss = criterion_N_with_AdvSup([output_change], [img_change], model.AdvSupResult, criterion)
            (loss / accumulate_steps).backward()
        else:
            with torch.cuda.amp.autocast():
                output_change = model(img1, img2)
                loss = criterion_N_with_AdvSup([output_change], [img_change], model.AdvSupResult, criterion)
            scaler.scale(loss / accumulate_steps).backward()
        
        if (batch_idx + 1) % accumulate_steps == 0:
            if scaler is None:
                optimizer.step()
                optimizer.zero_grad()
            else:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        
        output_change = F.softmax(output_change, dim=1)
        confusionMatrix(epochMetrics, [output_change[:, 1, :, :].detach()], [img_change.detach()])
        epochMetrics["Loss"][0] += loss.detach().cpu().item()
    
    epochMetrics["Loss"][0] = round(epochMetrics["Loss"][0] / len(trainDataLoader), 5)
    calculateMatrix(epochMetrics)
    printTable(epochMetrics, epochsDisplayMetrics, labelNameList)
    returnMetrics = {key: epochMetrics[key][-1] for key in plot_metrics}
    return returnMetrics


def validate(model, index, valDataLoader, criterion, scaler, minival=False):
    showRowLen = num_classes + 1
    epochMetricsList = ["TP", "FP", "TN", "FN", "Acc", "Pre", "Rec", "IoU", "TNR", "F1", "Kappa", "Loss"]
    epochMetrics = {key: [0] * showRowLen for key in epochMetricsList}
    
    dir_fold = result_dir + os.sep + str(index)
    model.eval()
    
    with torch.no_grad():
        for batch_idx, (img1, img2, label1, label2, img_change, dir_name) in tqdm(enumerate(valDataLoader)):
            img1, img2, img_change = img1.float(), img2.float(), img_change.float()
            
            if use_cuda:
                img1 = img1.to(device)
                img2 = img2.to(device)
                img_change = img_change.to(device)
            
            if scaler is None:
                output_change = model(img1, img2)
                loss = criterion_N_with_AdvSup([output_change], [img_change], model.AdvSupResult, criterion)
            else:
                with torch.cuda.amp.autocast():
                    output_change = model(img1, img2)
                    loss = criterion_N_with_AdvSup([output_change], [img_change], model.AdvSupResult, criterion)
            
            output_change = F.softmax(output_change, dim=1)
            confusionMatrix(epochMetrics, [output_change[:, 1, :, :].detach()], [img_change.detach()])
            epochMetrics["Loss"][0] += loss.detach().cpu().item()
            
            result_change = [result_dir + os.sep + str(index) + os.sep + i + '_change.png' for i in dir_name]
            output_change = output_change[:, 1, :, :].detach().float()
            output_change[output_change >= 0.5] = 255
            output_change[output_change < 0.5] = 0
            output_change = output_change.cpu().numpy()
    
    epochMetrics["Loss"][0] = round(epochMetrics["Loss"][0] / len(valDataLoader), 5)
    calculateMatrix(epochMetrics)
    printTable(epochMetrics, epochsDisplayMetrics, labelNameList)
    returnMetrics = {key: epochMetrics[key][-1] for key in plot_metrics}
    return returnMetrics


def criterion_N_with_AdvSup(pred, label, AdvSupResult, criterion):
    count = 0
    loss = 0.0
    for name, func in criterion.items():
        if func is not None:
            count += 1
            loss_temp = 0.0
            if name == "focalloss":
                loss_temp = func(pred[0], label[0].long())
            elif name == "adversialsupervised" and len(AdvSupResult) != 0:
                batch_size = label[0].size(0)
                for i in AdvSupResult:
                    unchange = func(i[:, 0], 1 - label[0].view(batch_size, -1).mean(dim=1)).mean()
                    change = func(i[:, 1], label[0].view(batch_size, -1).mean(dim=1)).mean()
                    loss_temp += (unchange + change) / 2
                loss_temp = loss_temp / len(AdvSupResult)
                loss_temp = 0.1 * loss_temp
            elif name == "bce":
                loss_temp = func(pred[0], label[0].long()).mean()
            elif name == "diceloss":
                loss_temp = 0.1 * func(pred[0][:, 1, :, :], label[0]).mean()
            loss += loss_temp
    return loss / count if count > 0 else loss


def criterion_N(pred, label, criterion):
    loss_c = criterion[0](pred[0].permute(0, 2, 3, 1).reshape(-1, 2), label[0].reshape(-1).long()).mean()
    for i in range(1, 2):
        loss_c += criterion[i](pred[0], label[0].long())
    loss = loss_c
    return loss / len(criterion)


def confusionMatrix(metrics, pred, label, threshold=0.5):
    for i in range(len(label)):
        singlePred = (pred[i] >= threshold).byte()
        singleLabel = (label[i] > threshold).byte()
        plus = singlePred + singleLabel
        FN = (singlePred < singleLabel).sum()
        FP = (singlePred > singleLabel).sum()
        TP = (plus == 2).sum()
        TN = (plus == 0).sum()
        
        metrics["TN"][i] = metrics["TN"][i] + TN.cpu().item()
        metrics["FP"][i] = metrics["FP"][i] + FP.cpu().item()
        metrics["FN"][i] = metrics["FN"][i] + FN.cpu().item()
        metrics["TP"][i] = metrics["TP"][i] + TP.cpu().item()
    return


def calculateMatrix(metrics):
    for i in range(len(metrics["Acc"]) - 1):
        TN = metrics["TN"][i]
        FP = metrics["FP"][i]
        FN = metrics["FN"][i]
        TP = metrics["TP"][i]
        
        metrics["Acc"][i] = (TP + TN) / (TP + TN + FP + FN)
        metrics["Pre"][i] = round(TP / (TP + FP + 0.0001) * 100, 3)
        metrics["Rec"][i] = round(TP / (TP + FN + 0.0001) * 100, 3)
        metrics["IoU"][i] = round(TP / (TP + FP + FN) * 100, 3)
        metrics["TNR"][i] = round(TN / (TN + FP) * 100, 3)
        metrics["F1"][i] = round(2 * TP / (2 * TP + FP + FN) * 100, 3)
        
        Pe = ((TP + FP) * (TP + FN) + (TN + FN) * (TN + FP)) / ((TP + FP + TN + FN) ** 2)
        metrics["Kappa"][i] = round((metrics["Acc"][i] - Pe) / (1 - Pe) * 100, 3)
        metrics["Acc"][i] = round(metrics["Acc"][i] * 100, 3)
    
    for k, v in metrics.items():
        if len(metrics["Acc"]) > 1:
            metrics[k][-1] = sum(metrics[k][:-1]) / (len(metrics["Acc"]) - 1)


def printTable(metrics, displayMetrics, labelName):
    labelNameCopy = labelName + ["average"]
    table = PrettyTable([""] + displayMetrics)
    for i in range(len(metrics["Acc"]) - 1):
        row = [labelNameCopy[i]] + [metrics[key][i] for key in displayMetrics]
        table.add_row(row)
    print(table)


def saveMetricsPlot(metrics, plot_metrics, stage):
    stage_name = list(metrics.keys())
    stage_name.sort()
    epochs = len(list(metrics.values())[0])
    
    # Draw plot
    x = range(epochs)
    len_stage = 1
    len_metric = len(plot_metrics)
    fig, axs = plt.subplots(len_metric, len_stage, dpi=600, figsize=(10, 10))
    
    if len_metric == 1:
        axs = [axs]
    
    for index, each_subplot in enumerate(plot_metrics):
        if each_subplot == "Loss":
            axs[index].set_ylim(0, 0.1)
        else:
            axs[index].set_ylim(0, 100)
        
        color = ['b', 'r', 'g', 'c', 'm', 'y', 'k', 'w']
        for each_stage in stage:
            axs[index].plot(x, metrics[each_subplot + " " + each_stage], label=each_stage)
        axs[index].set_title(each_subplot)
        axs[index].legend()
    
    fig.savefig(save_dir + os.sep + "results.png", bbox_inches="tight")
    plt.close("all")


def creatPlotProcess(metric_record, plot_metrics, stage):
    plotProcess = Process(target=saveMetricsPlot, args=(metric_record, plot_metrics, stage))
    plotProcess.start()
    plotProcess.join()


def main():
    setup_seed(42)
    model = get_cdnext(out_channels=2, backbone_scale=backboneName, 
                      pretrained=isPretrained, backbone_trainable=True).cuda()
    modelParams = filter(lambda p: p.requires_grad, model.parameters())
    print(model)
    model.to(device)
    model.zero_grad()
    
    optimizer = optim.AdamW(modelParams, lr=learning_rate, weight_decay=0.05, amsgrad=True)
    
    if use_amp == True:
        scaler = GradScaler()
    else:
        scaler = None
    
    trainDataset, trainDataLoader = data_loader(ROOTDIR, mode="train", taskList=labelNameList,
                                               miniScale=1, batch_size=BATCH_SIZE,
                                               shuffle=True, drop_last=True, 
                                               total_fold=5, valid_fold=5)
    
    valDataset, valDataloader = data_loader(ROOTDIR, mode="val", taskList=labelNameList, 
                                          miniScale=1, batch_size=BATCH_SIZE * 2, 
                                          shuffle=False, drop_last=False,
                                          total_fold=5, valid_fold=5)
    
    if lossType == "balance ce":
        criterion = {
            "focalloss": None,  # FocalLoss().to(device),
            "adversialsupervised": None,  # nn.L1Loss().to(device),
            "bce": nn.CrossEntropyLoss().to(device),  # None,
            "diceloss": None,  # DiceLoss().to(device),
        }
    
    start_i = 0
    metric_record = {(plot_metrics[i // len(stage)] + " " + stage[i % len(stage)]): [] 
                    for i in range(len(stage) * len(plot_metrics))}
    
    for i in range(start_i, total_epoch + 1):
        print(f" ====> epoch: {i}, learning:{optimizer.state_dict()['param_groups'][0]['lr']:.7f}, train and valid metrics: ")
        
        train_avg_metric = train(model, optimizer, i, trainDataLoader, criterion, scaler, trainDataset)
        for key in train_avg_metric.keys():
            metric_record[key + " train"].append(train_avg_metric[key])
        
        val_avg_metric = validate(model, i, valDataloader, criterion, scaler, minival=False)
        for key in val_avg_metric.keys():
            metric_record[key + " val"].append(val_avg_metric[key])
        
        cp_filename = f"epoch-{i}_trainF1{train_avg_metric['F1']:.2f}_valF1{val_avg_metric['F1']:.2f}"
        save_checkpoints(model, cp_filename)
        
        creatPlotProcess(metric_record, plot_metrics, stage)


if __name__ == "__main__":
    main()

Writing /kaggle/working/train.py


In [38]:
# list_kaggle_working.py
import os
import json
from pathlib import Path

def get_tree(path: Path, max_depth: int = 4, _depth=0, include_hidden=False):
    """Return nested dict representing directory tree of `path` up to max_depth."""
    if _depth > max_depth:
        return None
    node = {"type": "dir" if path.is_dir() else "file", "name": path.name}
    if path.is_dir():
        node["children"] = []
        try:
            entries = sorted(path.iterdir(), key=lambda p: (not p.is_dir(), p.name.lower()))
        except PermissionError:
            return {"type": "dir", "name": path.name, "error": "PermissionError"}
        for e in entries:
            if not include_hidden and e.name.startswith("."):
                continue
            child = get_tree(e, max_depth=max_depth, _depth=_depth+1, include_hidden=include_hidden)
            if child is not None:
                node["children"].append(child)
    else:
        node["size_bytes"] = path.stat().st_size
    return node

def print_tree(node, prefix=""):
    """Pretty-print the tree structure returned by get_tree."""
    if node is None:
        return
    marker = "📁" if node["type"] == "dir" else "📄"
    print(f"{prefix}{marker} {node['name']}", end="")
    if node["type"] == "file":
        print(f"  ({node.get('size_bytes', 0)} bytes)")
    else:
        if node.get("error"):
            print(f"  -- {node['error']}")
        else:
            print()
            for i, child in enumerate(node.get("children", [])):
                is_last = (i == len(node["children"]) - 1)
                new_prefix = prefix + ("    " if is_last else "│   ")
                print_tree(child, prefix + ("└── " if is_last else "├── "))

if __name__ == "__main__":
    root = Path("/kaggle/working")  # change if you want another path
    tree = get_tree(root, max_depth=3, include_hidden=False)  # adjust max_depth
    print_tree(tree)

    # optional: save JSON
    out_json = "/kaggle/working/working_tree.json"
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(tree, f, indent=2)
    print(f"\nSaved tree JSON to: {out_json}")


📁 working
├── 📁 data_utils
├── ├── 📁 __pycache__
├── ├── ├── 📄 __init__.cpython-311.pyc  (147 bytes)
├── ├── ├── 📄 augment.cpython-311.pyc  (7763 bytes)
├── ├── └── 📄 data_loader.cpython-311.pyc  (9932 bytes)
├── ├── 📄 __init__.py  (25 bytes)
├── ├── 📄 augment.py  (5501 bytes)
├── ├── 📄 data_loader.py  (6979 bytes)
├── └── 📄 slicingPatch.py  (10862 bytes)
├── 📁 models
├── ├── 📄 __init__.py  (646 bytes)
├── ├── 📄 cdnext.py  (10655 bytes)
├── ├── 📄 convnext.py  (13504 bytes)
├── ├── 📄 layers.py  (18834 bytes)
├── └── 📄 resnet.py  (15608 bytes)
├── 📁 processed_data
├── ├── 📁 test
├── ├── ├── 📁 label
├── ├── ├── 📁 T1
├── ├── └── 📁 T2
├── ├── 📁 train
├── ├── ├── 📁 label
├── ├── ├── 📁 T1
├── ├── └── 📁 T2
├── └── 📁 val
├── └── ├── 📁 label
├── └── ├── 📁 T1
├── └── └── 📁 T2
├── 📄 setup_directories.py  (1441 bytes)
└── 📄 train.py  (13976 bytes)

Saved tree JSON to: /kaggle/working/working_tree.json


In [39]:
%%writefile /kaggle/working/eval.py

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import GradScaler
from torch.autograd import Variable
from torch.utils import data as data_
from tqdm import tqdm
from models.cdnext import get_cdnext
from PIL import Image
from data_utils.data_loader import WSIDataset
import matplotlib.pyplot as plt
from multiprocessing import Process
from prettytable import PrettyTable
import glob
import random

labelNameList = ["change"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path setting - updated for Kaggle structure
ROOTDIR = {
    "LEVIRCD": "/kaggle/working/processed_data",
}

backboneName = "tiny"  # 'tiny','small','base','resnet18'
SIMULATE_BATCH_SIZE = 32
BATCH_SIZE = 16
accumulate_steps = SIMULATE_BATCH_SIZE // BATCH_SIZE
train_num_workers = 2
test_num_workers = 2
saveDatasetName = "-".join(ROOTDIR.keys())
save_dir = "checkpoint/" + saveDatasetName
total_epoch = 400
use_cuda = True
num_classes = len(labelNameList)
result_dir = "results/" + saveDatasetName
lossType = "balance ce"  # "ce" means cross entropy, "focal" means focal loss, "balance ce" means no parameters ce
learning_rate = 4e-5
itersDisplayMetrics = ["Acc", "Pre", "Rec", "IoU", "TNR", "Loss"]
epochsDisplayMetrics = ["Acc", "Pre", "Rec", "IoU", "TNR", "F1", "Kappa", "Loss"]
plot_metrics = ["F1", "IoU", "Loss"]
plot_metrics.sort()
stage = ["train", "val"]
stage.sort()
use_amp = True
model_path = "checkpoint/LEVIRCD/*.pth"  # Updated for Kaggle structure


def setup_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    # torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def data_loader(ROOTDIR, mode="train", taskList=labelNameList,
                miniScale=1, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, 
                total_fold=5, valid_fold=5):
    dataset = WSIDataset(root_dir=ROOTDIR, mode=mode, taskList=taskList, 
                        total_fold=total_fold, valid_fold=valid_fold, miniScale=miniScale)
    dataloader = data_.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=train_num_workers,
        shuffle=shuffle,
        drop_last=drop_last,
        pin_memory=True  # Load into memory to improve speed
    )
    return dataset, dataloader


def save_checkpoints(model, step):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    filename = "CDNet_" + str(step) + "_" + "-".join(labelNameList) + ".pth"
    torch.save(model.state_dict(), os.path.join(save_dir, filename))
    print(f" Save checkpoint {step.split('_')[0]} to {filename}. \n")


def validate(model, index, valDataLoader, criterion, scaler, minival=False):
    showRowLen = num_classes + 1
    epochMetricsList = ["TP", "FP", "TN", "FN", "Acc", "Pre", "Rec", "IoU", "TNR", "F1", "Kappa", "Loss"]
    epochMetrics = {key: [0] * showRowLen for key in epochMetricsList}
    
    dir_fold = result_dir + os.sep + str(index)
    if not os.path.exists(dir_fold):
        os.makedirs(dir_fold, exist_ok=True)
    
    model.eval()
    
    with torch.no_grad():
        for batch_idx, (img1, img2, label1, label2, img_change, dir_name) in tqdm(enumerate(valDataLoader)):
            img1, img2, img_change = img1.float(), img2.float(), img_change.float()
            
            if use_cuda:
                img1 = img1.to(device)
                img2 = img2.to(device)
                img_change = img_change.to(device)
            
            # Reset gradients
            if scaler is None:
                output_change = model(img1, img2)
                loss = criterion_N_with_AdvSup([output_change], [img_change], model.AdvSupResult, criterion)
            else:
                with torch.cuda.amp.autocast():
                    output_change = model(img1, img2)
                    loss = criterion_N_with_AdvSup([output_change], [img_change], model.AdvSupResult, criterion)
            
            output_change = F.softmax(output_change, dim=1)
            confusionMatrix(epochMetrics, [output_change[:, 1, :, :].detach()], [img_change.detach()])
            epochMetrics["Loss"][0] += loss.detach().cpu().item()
            
            result_change = [result_dir + os.sep + str(index) + os.sep + i + '_change.png' for i in dir_name]
            output_change = output_change[:, 1, :, :].detach().float()
            output_change[output_change >= 0.5] = 255
            output_change[output_change < 0.5] = 0
            output_change = output_change.cpu().numpy()
            
            # Save prediction images
            for i in range(output_change.shape[0]):
                temp_output_change = Image.fromarray(output_change[i].astype(np.uint8))
                temp_output_change.convert("RGB").save(result_change[i])
    
    epochMetrics["Loss"][0] = round(epochMetrics["Loss"][0] / len(valDataLoader), 5)
    calculateMatrix(epochMetrics)
    printTable(epochMetrics, epochsDisplayMetrics, labelNameList)
    returnMetrics = {key: epochMetrics[key][-1] for key in plot_metrics}
    return returnMetrics


def criterion_N_with_AdvSup(pred, label, AdvSupResult, criterion):
    count = 0
    loss = 0.0
    for name, func in criterion.items():
        if func is not None:
            count += 1
            loss_temp = 0.0
            if name == "focalloss":
                loss_temp = func(pred[0], label[0].long())
            elif name == "adversialsupervised" and len(AdvSupResult) != 0:
                batch_size = label[0].size(0)
                for i in AdvSupResult:
                    unchange = func(i[:, 0], 1 - label[0].view(batch_size, -1).mean(dim=1)).mean()
                    change = func(i[:, 1], label[0].view(batch_size, -1).mean(dim=1)).mean()
                    loss_temp += (unchange + change) / 2
                loss_temp = loss_temp / len(AdvSupResult)
                loss_temp = 0.1 * loss_temp
            elif name == "bce":
                loss_temp = func(pred[0], label[0].long()).mean()
            elif name == "diceloss":
                loss_temp = 0.1 * func(pred[0][:, 1, :, :], label[0]).mean()
            loss += loss_temp
    return loss / count if count > 0 else loss


def criterion_N(pred, label, criterion):
    loss_c = criterion[0](pred[0].permute(0, 2, 3, 1).reshape(-1, 2), label[0].reshape(-1).long()).mean()
    for i in range(1, 2):
        loss_c += criterion[i](pred[0], label[0].long())
    loss = loss_c
    return loss / len(criterion)


def confusionMatrix(metrics, pred, label, threshold=0.5):
    for i in range(len(label)):
        singlePred = (pred[i] >= threshold).byte()
        singleLabel = (label[i] > threshold).byte()
        plus = singlePred + singleLabel
        FN = (singlePred < singleLabel).sum()
        FP = (singlePred > singleLabel).sum()
        TP = (plus == 2).sum()
        TN = (plus == 0).sum()
        
        metrics["TN"][i] = metrics["TN"][i] + TN.cpu().item()
        metrics["FP"][i] = metrics["FP"][i] + FP.cpu().item()
        metrics["FN"][i] = metrics["FN"][i] + FN.cpu().item()
        metrics["TP"][i] = metrics["TP"][i] + TP.cpu().item()
    return


def calculateMatrix(metrics):
    for i in range(len(metrics["Acc"]) - 1):
        TN = metrics["TN"][i]
        FP = metrics["FP"][i]
        FN = metrics["FN"][i]
        TP = metrics["TP"][i]
        
        metrics["Acc"][i] = (TP + TN) / (TP + TN + FP + FN)
        metrics["Pre"][i] = round(TP / (TP + FP + 0.0001) * 100, 3)
        metrics["Rec"][i] = round(TP / (TP + FN + 0.0001) * 100, 3)
        metrics["IoU"][i] = round(TP / (TP + FP + FN) * 100, 3)
        metrics["TNR"][i] = round(TN / (TN + FP) * 100, 3)
        metrics["F1"][i] = round(2 * TP / (2 * TP + FP + FN) * 100, 3)
        
        Pe = ((TP + FP) * (TP + FN) + (TN + FN) * (TN + FP)) / ((TP + FP + TN + FN) ** 2)
        metrics["Kappa"][i] = round((metrics["Acc"][i] - Pe) / (1 - Pe) * 100, 3)
        metrics["Acc"][i] = round(metrics["Acc"][i] * 100, 3)
    
    for k, v in metrics.items():
        if len(metrics["Acc"]) > 1:
            metrics[k][-1] = sum(metrics[k][:-1]) / (len(metrics["Acc"]) - 1)


def printTable(metrics, displayMetrics, labelName):
    labelNameCopy = labelName + ["average"]
    table = PrettyTable([""] + displayMetrics)
    for i in range(len(metrics["Acc"]) - 1):
        row = [labelNameCopy[i]] + [metrics[key][i] for key in displayMetrics]
        table.add_row(row)
    print(table)


def saveMetricsPlot(metrics, plot_metrics, stage):
    stage_name = list(metrics.keys())
    stage_name.sort()
    epochs = len(list(metrics.values())[0])
    
    x = range(epochs)
    len_stage = 1
    len_metric = len(plot_metrics)
    fig, axs = plt.subplots(len_metric, len_stage, dpi=600, figsize=(10, 10))
    
    if len_metric == 1:
        axs = [axs]
    
    for index, each_subplot in enumerate(plot_metrics):
        if each_subplot == "Loss":
            axs[index].set_ylim(0, 0.05)
        else:
            axs[index].set_ylim(70, 100)
        
        color = ['b', 'r', 'g', 'c', 'm', 'y', 'k', 'w']
        for each_stage in stage:
            axs[index].plot(x, metrics[each_subplot + " " + each_stage], label=each_stage)
        axs[index].set_title(each_subplot)
        axs[index].legend()
    
    fig.savefig(save_dir + os.sep + "results.png", bbox_inches="tight")
    plt.close("all")


def creatPlotProcess(metric_record, plot_metrics, stage):
    plotProcess = Process(target=saveMetricsPlot, args=(metric_record, plot_metrics, stage))
    plotProcess.start()
    plotProcess.join()


def main():
    # Find all checkpoint models
    ckmodels = glob.glob(model_path)
    if not ckmodels:
        print(f"No checkpoint models found at: {model_path}")
        print("Please ensure you have trained models in the checkpoint directory.")
        return
    
    for model_file in ckmodels:
        print(f"Evaluating model: {model_file}")
        
        setup_seed(42)
        model = get_cdnext(out_channels=2, backbone_scale=backboneName, 
                          pretrained=True, backbone_trainable=True).cuda()
        modelParams = filter(lambda p: p.requires_grad, model.parameters())
        
        # Load the trained model weights
        try:
            model.load_state_dict(torch.load(model_file, map_location=device))
            print(f"Successfully loaded model from: {model_file}")
        except Exception as e:
            print(f"Error loading model {model_file}: {str(e)}")
            continue
            
        model.to(device).eval()
        
        optimizer = optim.AdamW(modelParams, lr=learning_rate, weight_decay=0.05, amsgrad=True)
        
        if use_amp == True:
            scaler = GradScaler()
        else:
            scaler = None
        
        # Load test dataset
        valDataset, valDataloader = data_loader(ROOTDIR, mode="test", taskList=labelNameList, 
                                              miniScale=1, batch_size=BATCH_SIZE * 2, 
                                              shuffle=False, drop_last=False,
                                              total_fold=5, valid_fold=5)
        
        if lossType == "balance ce":
            criterion = {
                "focalloss": None,  # FocalLoss().to(device),
                "adversialsupervised": None,  # nn.L1Loss().to(device),
                "bce": nn.CrossEntropyLoss().to(device),  # None,
                "diceloss": None,  # DiceLoss().to(device),
            }
        
        start_i = 0
        metric_record = {(plot_metrics[i // len(stage)] + " " + stage[i % len(stage)]): [] 
                        for i in range(len(stage) * len(plot_metrics))}
        
        print(f" ====> Evaluating model: {os.path.basename(model_file)}, learning: {optimizer.state_dict()['param_groups'][0]['lr']:.7f}")
        
        val_avg_metric = validate(model, 90, valDataloader, criterion, scaler, minival=False)
        for key in val_avg_metric.keys():
            metric_record[key + " val"].append(val_avg_metric[key])
        
        print(f"Evaluation completed for {os.path.basename(model_file)}")
        print("-" * 80)


def criterion_N_with_AdvSup(pred, label, AdvSupResult, criterion):
    count = 0
    loss = 0.0
    for name, func in criterion.items():
        if func is not None:
            count += 1
            loss_temp = 0.0
            if name == "focalloss":
                loss_temp = func(pred[0], label[0].long())
            elif name == "adversialsupervised" and len(AdvSupResult) != 0:
                batch_size = label[0].size(0)
                for i in AdvSupResult:
                    unchange = func(i[:, 0], 1 - label[0].view(batch_size, -1).mean(dim=1)).mean()
                    change = func(i[:, 1], label[0].view(batch_size, -1).mean(dim=1)).mean()
                    loss_temp += (unchange + change) / 2
                loss_temp = loss_temp / len(AdvSupResult)
                loss_temp = 0.1 * loss_temp
            elif name == "bce":
                loss_temp = func(pred[0], label[0].long()).mean()
            elif name == "diceloss":
                loss_temp = 0.1 * func(pred[0][:, 1, :, :], label[0]).mean()
            loss += loss_temp
    return loss / count if count > 0 else loss


def criterion_N(pred, label, criterion):
    loss_c = criterion[0](pred[0].permute(0, 2, 3, 1).reshape(-1, 2), label[0].reshape(-1).long()).mean()
    for i in range(1, 2):
        loss_c += criterion[i](pred[0], label[0].long())
    loss = loss_c
    return loss / len(criterion)


def confusionMatrix(metrics, pred, label, threshold=0.5):
    for i in range(len(label)):
        singlePred = (pred[i] >= threshold).byte()
        singleLabel = (label[i] > threshold).byte()
        plus = singlePred + singleLabel
        FN = (singlePred < singleLabel).sum()
        FP = (singlePred > singleLabel).sum()
        TP = (plus == 2).sum()
        TN = (plus == 0).sum()
        
        metrics["TN"][i] = metrics["TN"][i] + TN.cpu().item()
        metrics["FP"][i] = metrics["FP"][i] + FP.cpu().item()
        metrics["FN"][i] = metrics["FN"][i] + FN.cpu().item()
        metrics["TP"][i] = metrics["TP"][i] + TP.cpu().item()
    return


def calculateMatrix(metrics):
    for i in range(len(metrics["Acc"]) - 1):
        TN = metrics["TN"][i]
        FP = metrics["FP"][i]
        FN = metrics["FN"][i]
        TP = metrics["TP"][i]
        
        metrics["Acc"][i] = (TP + TN) / (TP + TN + FP + FN)
        metrics["Pre"][i] = round(TP / (TP + FP + 0.0001) * 100, 3)
        metrics["Rec"][i] = round(TP / (TP + FN + 0.0001) * 100, 3)
        metrics["IoU"][i] = round(TP / (TP + FP + FN) * 100, 3)
        metrics["TNR"][i] = round(TN / (TN + FP) * 100, 3)
        metrics["F1"][i] = round(2 * TP / (2 * TP + FP + FN) * 100, 3)
        
        Pe = ((TP + FP) * (TP + FN) + (TN + FN) * (TN + FP)) / ((TP + FP + TN + FN) ** 2)
        metrics["Kappa"][i] = round((metrics["Acc"][i] - Pe) / (1 - Pe) * 100, 3)
        metrics["Acc"][i] = round(metrics["Acc"][i] * 100, 3)
    
    for k, v in metrics.items():
        if len(metrics["Acc"]) > 1:
            metrics[k][-1] = sum(metrics[k][:-1]) / (len(metrics["Acc"]) - 1)


def printTable(metrics, displayMetrics, labelName):
    labelNameCopy = labelName + ["average"]
    table = PrettyTable([""] + displayMetrics)
    for i in range(len(metrics["Acc"]) - 1):
        row = [labelNameCopy[i]] + [metrics[key][i] for key in displayMetrics]
        table.add_row(row)
    print(table)


if __name__ == "__main__":
    main()

Writing /kaggle/working/eval.py


In [40]:
# In your Kaggle notebook
%cd /kaggle/working
!python train.py

/kaggle/working
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/kaggle/working/models/convnext.py:168: UserWarning: Overwriting convnext_tiny in registry with models.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/kaggle/working/models/convnext.py:185: UserWarning: Overwriting convnext_small in registry with models.convnext.convnext_small. This is because the name being registered conflicts w